<a href="https://colab.research.google.com/github/Rahad31/Kl-FedDis-Research-/blob/main/ordinary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import nn
from typing import Dict

CPU times: user 377 µs, sys: 0 ns, total: 377 µs
Wall time: 391 µs


In [ ]:
# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [ ]:
# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    for epoch in range(epochs):
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()


In [ ]:

# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:06<00:00, 27.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])


In [ ]:
# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)


In [ ]:
# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [ ]:
# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

In [ ]:
# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> float:
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

In [ ]:
def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {
        "mean": vae.encoder[-1].bias.data.cpu().numpy(),
        "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
    }

    return distribution_info

In [ ]:
def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass


In [ ]:
# Define logic to generate augmented data using Ordinary Normal distribution
def generate_augmented_data(vae: VAE, distribution_info: Dict) -> torch.Tensor:
    # Generate augmented data using Ordinary Normal distribution
    mean = distribution_info["mean"]
    std = distribution_info["std"]
    augmented_data = torch.randn(64, vae.z_dim) * std + mean
    return augmented_data

In [ ]:
def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info)

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

In [ ]:
# Define logic to receive distribution information from global server
def receive_distribution_info() -> Dict:
    # Receive distribution information logic
    distribution_info = {
        "mean": np.zeros(20),  # Adjust the size based on your latent space dimension
        "std": np.ones(20)
    }
    return distribution_info

In [ ]:
def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

In [ ]:
%%time

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=5)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()


Epoch [1/10], Training Loss: 2.304, Validation Accuracy: 10.23%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 10.23%
Epoch [3/10], Training Loss: 2.303, Validation Accuracy: 10.23%
Epoch [4/10], Training Loss: 2.303, Validation Accuracy: 10.23%
Epoch [5/10], Training Loss: 2.302, Validation Accuracy: 10.23%
Epoch [6/10], Training Loss: 2.302, Validation Accuracy: 10.23%
Epoch [7/10], Training Loss: 2.301, Validation Accuracy: 10.23%
Epoch [8/10], Training Loss: 2.301, Validation Accuracy: 10.23%
Epoch [9/10], Training Loss: 2.300, Validation Accuracy: 10.23%
Epoch [10/10], Training Loss: 2.300, Validation Accuracy: 10.23%
Epoch [1/10], Training Loss: 2.300, Validation Accuracy: 10.25%
Epoch [2/10], Training Loss: 2.299, Validation Accuracy: 11.46%
Epoch [3/10], Training Loss: 2.298, Validation Accuracy: 13.78%
Epoch [4/10], Training Loss: 2.297, Validation Accuracy: 15.52%
Epoch [5/10], Training Loss: 2.295, Validation Accuracy: 16.95%
Epoch [6/10], Training Loss: 2.293, Val

In [ ]:
from scipy.stats import truncnorm
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from typing import Dict, Tuple

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(epochs):
        vae.train()
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()

        scheduler.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)



# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")




# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> float:
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy




# Initialize clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {
        "mean": vae.encoder[-1].bias.data.cpu().numpy(),
        "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
    }

    return distribution_info

def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass


# Define logic to generate augmented data using Truncated Normal distribution
def generate_augmented_data(vae: VAE, distribution_info: Dict) -> torch.Tensor:
    # Generate augmented data using Truncated Normal distribution
    mean = distribution_info["mean"]
    std = distribution_info["std"]
    a = (0 - mean) / std
    b = np.inf
    augmented_data = torch.from_numpy(truncnorm.rvs(a, b, loc=mean, scale=std, size=(64, vae.z_dim))).float()
    return augmented_data





def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info)

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())


def receive_distribution_info() -> Dict:
    # Implement the logic to receive the distribution information from the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to receive the information

    # Example implementation:
    # Receive the distribution information from the global server using a network protocol
    # For example, you can use the `socket` module to receive the information over a network
    # or use a message queue like `RabbitMQ` to receive the information
    distribution_info = {
        "mean": np.zeros(20),
        "std": np.ones(20)
    }
    return distribution_info

def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define global server procedure
# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()


In [ ]:
import re

# Your provided text
log = """
Epoch [1/10], Training Loss: 2.304, Validation Accuracy: 10.23%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.39%
Epoch [3/10], Training Loss: 2.302, Validation Accuracy: 10.40%
Epoch [4/10], Training Loss: 2.301, Validation Accuracy: 10.46%
Epoch [5/10], Training Loss: 2.300, Validation Accuracy: 10.51%
Epoch [6/10], Training Loss: 2.299, Validation Accuracy: 11.06%
Epoch [7/10], Training Loss: 2.297, Validation Accuracy: 11.38%
Epoch [8/10], Training Loss: 2.295, Validation Accuracy: 12.02%
Epoch [9/10], Training Loss: 2.293, Validation Accuracy: 12.71%
Epoch [10/10], Training Loss: 2.289, Validation Accuracy: 14.52%
Epoch [1/10], Training Loss: 2.286, Validation Accuracy: 17.72%
Epoch [2/10], Training Loss: 2.279, Validation Accuracy: 17.84%
Epoch [3/10], Training Loss: 2.267, Validation Accuracy: 16.83%
Epoch [4/10], Training Loss: 2.250, Validation Accuracy: 17.60%
Epoch [5/10], Training Loss: 2.228, Validation Accuracy: 19.17%
Epoch [6/10], Training Loss: 2.204, Validation Accuracy: 20.78%
Epoch [7/10], Training Loss: 2.176, Validation Accuracy: 22.38%
Epoch [8/10], Training Loss: 2.146, Validation Accuracy: 23.42%
Epoch [9/10], Training Loss: 2.110, Validation Accuracy: 24.32%
Epoch [10/10], Training Loss: 2.072, Validation Accuracy: 26.29%
Epoch [1/10], Training Loss: 2.046, Validation Accuracy: 26.32%
Epoch [2/10], Training Loss: 2.013, Validation Accuracy: 26.74%
Epoch [3/10], Training Loss: 1.985, Validation Accuracy: 27.73%
Epoch [4/10], Training Loss: 1.962, Validation Accuracy: 28.39%
Epoch [5/10], Training Loss: 1.943, Validation Accuracy: 28.82%
Epoch [6/10], Training Loss: 1.927, Validation Accuracy: 29.19%
Epoch [7/10], Training Loss: 1.908, Validation Accuracy: 29.31%
Epoch [8/10], Training Loss: 1.892, Validation Accuracy: 29.99%
Epoch [9/10], Training Loss: 1.874, Validation Accuracy: 30.69%
Epoch [10/10], Training Loss: 1.855, Validation Accuracy: 31.64%
Epoch [1/10], Training Loss: 1.850, Validation Accuracy: 32.97%
Epoch [2/10], Training Loss: 1.825, Validation Accuracy: 33.05%
Epoch [3/10], Training Loss: 1.806, Validation Accuracy: 34.13%
Epoch [4/10], Training Loss: 1.788, Validation Accuracy: 34.44%
Epoch [5/10], Training Loss: 1.766, Validation Accuracy: 34.67%
Epoch [6/10], Training Loss: 1.752, Validation Accuracy: 35.97%
Epoch [7/10], Training Loss: 1.734, Validation Accuracy: 36.63%
Epoch [8/10], Training Loss: 1.721, Validation Accuracy: 36.75%
Epoch [9/10], Training Loss: 1.703, Validation Accuracy: 37.37%
Epoch [10/10], Training Loss: 1.687, Validation Accuracy: 38.02%
Epoch [1/10], Training Loss: 1.712, Validation Accuracy: 37.85%
Epoch [2/10], Training Loss: 1.694, Validation Accuracy: 39.17%
Epoch [3/10], Training Loss: 1.688, Validation Accuracy: 39.35%
Epoch [4/10], Training Loss: 1.666, Validation Accuracy: 39.82%
Epoch [5/10], Training Loss: 1.660, Validation Accuracy: 39.85%
Epoch [6/10], Training Loss: 1.644, Validation Accuracy: 40.52%
Epoch [7/10], Training Loss: 1.629, Validation Accuracy: 41.12%
Epoch [8/10], Training Loss: 1.619, Validation Accuracy: 40.65%
Epoch [9/10], Training Loss: 1.611, Validation Accuracy: 41.61%
Epoch [10/10], Training Loss: 1.602, Validation Accuracy: 42.00%
Epoch [1/10], Training Loss: 1.618, Validation Accuracy: 42.16%
Epoch [2/10], Training Loss: 1.603, Validation Accuracy: 41.07%
Epoch [3/10], Training Loss: 1.600, Validation Accuracy: 42.26%
Epoch [4/10], Training Loss: 1.582, Validation Accuracy: 42.12%
Epoch [5/10], Training Loss: 1.576, Validation Accuracy: 43.30%
Epoch [6/10], Training Loss: 1.568, Validation Accuracy: 43.31%
Epoch [7/10], Training Loss: 1.555, Validation Accuracy: 43.59%
Epoch [8/10], Training Loss: 1.545, Validation Accuracy: 44.72%
Epoch [9/10], Training Loss: 1.540, Validation Accuracy: 43.08%
Epoch [10/10], Training Loss: 1.531, Validation Accuracy: 44.20%
Epoch [1/10], Training Loss: 1.530, Validation Accuracy: 44.49%
Epoch [2/10], Training Loss: 1.522, Validation Accuracy: 44.97%
Epoch [3/10], Training Loss: 1.506, Validation Accuracy: 44.74%
Epoch [4/10], Training Loss: 1.496, Validation Accuracy: 45.18%
Epoch [5/10], Training Loss: 1.484, Validation Accuracy: 45.09%
Epoch [6/10], Training Loss: 1.481, Validation Accuracy: 45.35%
Epoch [7/10], Training Loss: 1.469, Validation Accuracy: 45.75%
Epoch [8/10], Training Loss: 1.460, Validation Accuracy: 45.42%
Epoch [9/10], Training Loss: 1.454, Validation Accuracy: 46.19%
Epoch [10/10], Training Loss: 1.445, Validation Accuracy: 46.37%
Epoch [1/10], Training Loss: 1.484, Validation Accuracy: 46.75%
Epoch [2/10], Training Loss: 1.480, Validation Accuracy: 46.40%
Epoch [3/10], Training Loss: 1.465, Validation Accuracy: 47.26%
Epoch [4/10], Training Loss: 1.454, Validation Accuracy: 46.66%
Epoch [5/10], Training Loss: 1.445, Validation Accuracy: 47.40%
Epoch [6/10], Training Loss: 1.439, Validation Accuracy: 47.26%
Epoch [7/10], Training Loss: 1.433, Validation Accuracy: 46.78%
Epoch [8/10], Training Loss: 1.422, Validation Accuracy: 47.24%
Epoch [9/10], Training Loss: 1.418, Validation Accuracy: 47.41%
Epoch [10/10], Training Loss: 1.412, Validation Accuracy: 47.62%
Epoch [1/10], Training Loss: 1.436, Validation Accuracy: 48.26%
Epoch [2/10], Training Loss: 1.429, Validation Accuracy: 47.29%
Epoch [3/10], Training Loss: 1.416, Validation Accuracy: 48.46%
Epoch [4/10], Training Loss: 1.411, Validation Accuracy: 47.92%
Epoch [5/10], Training Loss: 1.402, Validation Accuracy: 48.90%
Epoch [6/10], Training Loss: 1.392, Validation Accuracy: 48.59%
Epoch [7/10], Training Loss: 1.383, Validation Accuracy: 48.00%
Epoch [8/10], Training Loss: 1.377, Validation Accuracy: 48.68%
Epoch [9/10], Training Loss: 1.373, Validation Accuracy: 48.55%
Epoch [10/10], Training Loss: 1.359, Validation Accuracy: 48.74%
Epoch [1/10], Training Loss: 1.417, Validation Accuracy: 49.03%
Epoch [2/10], Training Loss: 1.415, Validation Accuracy: 49.57%
Epoch [3/10], Training Loss: 1.398, Validation Accuracy: 49.46%
Epoch [4/10], Training Loss: 1.390, Validation Accuracy: 49.47%
Epoch [5/10], Training Loss: 1.380, Validation Accuracy: 49.49%
Epoch [6/10], Training Loss: 1.368, Validation Accuracy: 50.03%
Epoch [7/10], Training Loss: 1.365, Validation Accuracy: 49.50%
Epoch [8/10], Training Loss: 1.356, Validation Accuracy: 49.78%
Epoch [9/10], Training Loss: 1.351, Validation Accuracy: 50.29%
Epoch [10/10], Training Loss: 1.343, Validation Accuracy: 50.01%
Epoch [1/10], Training Loss: 1.397, Validation Accuracy: 50.54%
Epoch [2/10], Training Loss: 1.376, Validation Accuracy: 49.81%
Epoch [3/10], Training Loss: 1.366, Validation Accuracy: 50.71%
Epoch [4/10], Training Loss: 1.351, Validation Accuracy: 51.03%
Epoch [5/10], Training Loss: 1.341, Validation Accuracy: 51.24%
Epoch [6/10], Training Loss: 1.333, Validation Accuracy: 50.52%
Epoch [7/10], Training Loss: 1.323, Validation Accuracy: 51.22%
Epoch [8/10], Training Loss: 1.316, Validation Accuracy: 51.07%
Epoch [9/10], Training Loss: 1.308, Validation Accuracy: 51.01%
Epoch [10/10], Training Loss: 1.313, Validation Accuracy: 50.84%
Epoch [1/10], Training Loss: 1.335, Validation Accuracy: 51.86%
Epoch [2/10], Training Loss: 1.311, Validation Accuracy: 51.89%
Epoch [3/10], Training Loss: 1.307, Validation Accuracy: 51.90%
Epoch [4/10], Training Loss: 1.290, Validation Accuracy: 52.18%
Epoch [5/10], Training Loss: 1.280, Validation Accuracy: 51.86%
Epoch [6/10], Training Loss: 1.270, Validation Accuracy: 52.50%
Epoch [7/10], Training Loss: 1.267, Validation Accuracy: 52.05%
Epoch [8/10], Training Loss: 1.255, Validation Accuracy: 50.80%
Epoch [9/10], Training Loss: 1.248, Validation Accuracy: 52.27%
Epoch [10/10], Training Loss: 1.234, Validation Accuracy: 52.33%
Epoch [1/10], Training Loss: 1.323, Validation Accuracy: 52.94%
Epoch [2/10], Training Loss: 1.303, Validation Accuracy: 52.61%
Epoch [3/10], Training Loss: 1.294, Validation Accuracy: 52.37%
Epoch [4/10], Training Loss: 1.284, Validation Accuracy: 52.04%
Epoch [5/10], Training Loss: 1.272, Validation Accuracy: 52.91%
Epoch [6/10], Training Loss: 1.258, Validation Accuracy: 52.78%
Epoch [7/10], Training Loss: 1.256, Validation Accuracy: 52.36%
Epoch [8/10], Training Loss: 1.240, Validation Accuracy: 53.43%
Epoch [9/10], Training Loss: 1.235, Validation Accuracy: 53.43%
Epoch [10/10], Training Loss: 1.227, Validation Accuracy: 53.06%
Epoch [1/10], Training Loss: 1.277, Validation Accuracy: 53.18%
Epoch [2/10], Training Loss: 1.268, Validation Accuracy: 53.77%
Epoch [3/10], Training Loss: 1.249, Validation Accuracy: 53.35%
Epoch [4/10], Training Loss: 1.235, Validation Accuracy: 53.48%
Epoch [5/10], Training Loss: 1.224, Validation Accuracy: 53.38%
Epoch [6/10], Training Loss: 1.219, Validation Accuracy: 53.05%
Epoch [7/10], Training Loss: 1.209, Validation Accuracy: 53.40%
Epoch [8/10], Training Loss: 1.197, Validation Accuracy: 53.66%
Epoch [9/10], Training Loss: 1.197, Validation Accuracy: 53.54%
Epoch [10/10], Training Loss: 1.180, Validation Accuracy: 54.20%
Epoch [1/10], Training Loss: 1.272, Validation Accuracy: 53.92%
Epoch [2/10], Training Loss: 1.246, Validation Accuracy: 53.26%
Epoch [3/10], Training Loss: 1.234, Validation Accuracy: 54.00%
Epoch [4/10], Training Loss: 1.221, Validation Accuracy: 54.25%
Epoch [5/10], Training Loss: 1.211, Validation Accuracy: 53.40%
Epoch [6/10], Training Loss: 1.201, Validation Accuracy: 54.27%
Epoch [7/10], Training Loss: 1.190, Validation Accuracy: 53.64%
Epoch [8/10], Training Loss: 1.178, Validation Accuracy: 54.58%
Epoch [9/10], Training Loss: 1.174, Validation Accuracy: 54.28%
Epoch [10/10], Training Loss: 1.162, Validation Accuracy: 54.55%
Epoch [1/10], Training Loss: 1.253, Validation Accuracy: 54.30%
Epoch [2/10], Training Loss: 1.229, Validation Accuracy: 55.39%
Epoch [3/10], Training Loss: 1.205, Validation Accuracy: 55.22%
Epoch [4/10], Training Loss: 1.190, Validation Accuracy: 55.29%
Epoch [5/10], Training Loss: 1.178, Validation Accuracy: 54.05%
Epoch [6/10], Training Loss: 1.172, Validation Accuracy: 55.60%
Epoch [7/10], Training Loss: 1.153, Validation Accuracy: 55.25%
Epoch [8/10], Training Loss: 1.155, Validation Accuracy: 54.18%
Epoch [9/10], Training Loss: 1.146, Validation Accuracy: 55.79%
Epoch [10/10], Training Loss: 1.131, Validation Accuracy: 55.39%
Epoch [1/10], Training Loss: 1.196, Validation Accuracy: 55.35%
Epoch [2/10], Training Loss: 1.182, Validation Accuracy: 55.93%
Epoch [3/10], Training Loss: 1.160, Validation Accuracy: 56.11%
Epoch [4/10], Training Loss: 1.143, Validation Accuracy: 56.15%
Epoch [5/10], Training Loss: 1.130, Validation Accuracy: 56.20%
Epoch [6/10], Training Loss: 1.123, Validation Accuracy: 55.83%
Epoch [7/10], Training Loss: 1.114, Validation Accuracy: 56.08%
Epoch [8/10], Training Loss: 1.098, Validation Accuracy: 56.06%
Epoch [9/10], Training Loss: 1.091, Validation Accuracy: 56.10%
Epoch [10/10], Training Loss: 1.085, Validation Accuracy: 54.38%
Epoch [1/10], Training Loss: 1.212, Validation Accuracy: 56.58%
Epoch [2/10], Training Loss: 1.176, Validation Accuracy: 56.47%
Epoch [3/10], Training Loss: 1.170, Validation Accuracy: 56.87%
Epoch [4/10], Training Loss: 1.154, Validation Accuracy: 56.70%
Epoch [5/10], Training Loss: 1.140, Validation Accuracy: 56.62%
Epoch [6/10], Training Loss: 1.125, Validation Accuracy: 56.90%
Epoch [7/10], Training Loss: 1.109, Validation Accuracy: 56.71%
Epoch [8/10], Training Loss: 1.108, Validation Accuracy: 56.52%
Epoch [9/10], Training Loss: 1.093, Validation Accuracy: 56.13%
Epoch [10/10], Training Loss: 1.086, Validation Accuracy: 56.61%
Epoch [1/10], Training Loss: 1.156, Validation Accuracy: 56.98%
Epoch [2/10], Training Loss: 1.136, Validation Accuracy: 56.16%
Epoch [3/10], Training Loss: 1.117, Validation Accuracy: 57.32%
Epoch [4/10], Training Loss: 1.100, Validation Accuracy: 56.86%
Epoch [5/10], Training Loss: 1.093, Validation Accuracy: 57.18%
Epoch [6/10], Training Loss: 1.081, Validation Accuracy: 57.17%
Epoch [7/10], Training Loss: 1.073, Validation Accuracy: 57.05%
Epoch [8/10], Training Loss: 1.056, Validation Accuracy: 56.92%
Epoch [9/10], Training Loss: 1.053, Validation Accuracy: 57.95%
Epoch [10/10], Training Loss: 1.037, Validation Accuracy: 56.62%
Epoch [1/10], Training Loss: 1.162, Validation Accuracy: 58.08%
Epoch [2/10], Training Loss: 1.131, Validation Accuracy: 57.04%
Epoch [3/10], Training Loss: 1.116, Validation Accuracy: 56.60%
Epoch [4/10], Training Loss: 1.100, Validation Accuracy: 57.53%
Epoch [5/10], Training Loss: 1.086, Validation Accuracy: 57.85%
Epoch [6/10], Training Loss: 1.085, Validation Accuracy: 56.60%
Epoch [7/10], Training Loss: 1.057, Validation Accuracy: 57.85%
Epoch [8/10], Training Loss: 1.061, Validation Accuracy: 57.37%
Epoch [9/10], Training Loss: 1.036, Validation Accuracy: 57.73%
Epoch [10/10], Training Loss: 1.027, Validation Accuracy: 57.26%
Epoch [1/10], Training Loss: 1.136, Validation Accuracy: 57.98%
Epoch [2/10], Training Loss: 1.114, Validation Accuracy: 57.94%
Epoch [3/10], Training Loss: 1.089, Validation Accuracy: 57.44%
Epoch [4/10], Training Loss: 1.076, Validation Accuracy: 58.03%
Epoch [5/10], Training Loss: 1.055, Validation Accuracy: 57.91%
Epoch [6/10], Training Loss: 1.040, Validation Accuracy: 58.25%
Epoch [7/10], Training Loss: 1.029, Validation Accuracy: 58.16%
Epoch [8/10], Training Loss: 1.028, Validation Accuracy: 58.38%
Epoch [9/10], Training Loss: 1.007, Validation Accuracy: 57.65%
Epoch [10/10], Training Loss: 1.003, Validation Accuracy: 58.61%
Epoch [1/10], Training Loss: 1.107, Validation Accuracy: 56.73%
Epoch [2/10], Training Loss: 1.073, Validation Accuracy: 57.65%
Epoch [3/10], Training Loss: 1.060, Validation Accuracy: 58.57%
Epoch [4/10], Training Loss: 1.034, Validation Accuracy: 58.43%
Epoch [5/10], Training Loss: 1.026, Validation Accuracy: 58.48%
Epoch [6/10], Training Loss: 1.011, Validation Accuracy: 58.69%
Epoch [7/10], Training Loss: 0.996, Validation Accuracy: 58.49%
Epoch [8/10], Training Loss: 0.979, Validation Accuracy: 58.67%
Epoch [9/10], Training Loss: 0.969, Validation Accuracy: 58.06%
Epoch [10/10], Training Loss: 0.958, Validation Accuracy: 57.54%
Epoch [1/10], Training Loss: 1.121, Validation Accuracy: 58.58%
Epoch [2/10], Training Loss: 1.078, Validation Accuracy: 58.84%
Epoch [3/10], Training Loss: 1.063, Validation Accuracy: 58.49%
Epoch [4/10], Training Loss: 1.037, Validation Accuracy: 58.59%
Epoch [5/10], Training Loss: 1.027, Validation Accuracy: 58.61%
Epoch [6/10], Training Loss: 1.012, Validation Accuracy: 58.33%
Epoch [7/10], Training Loss: 1.000, Validation Accuracy: 58.84%
Epoch [8/10], Training Loss: 0.982, Validation Accuracy: 58.53%
Epoch [9/10], Training Loss: 0.975, Validation Accuracy: 59.15%
Epoch [10/10], Training Loss: 0.967, Validation Accuracy: 58.63%
Epoch [1/10], Training Loss: 1.083, Validation Accuracy: 58.13%
Epoch [2/10], Training Loss: 1.047, Validation Accuracy: 58.83%
Epoch [3/10], Training Loss: 1.017, Validation Accuracy: 59.03%
Epoch [4/10], Training Loss: 0.999, Validation Accuracy: 59.10%
Epoch [5/10], Training Loss: 0.987, Validation Accuracy: 58.39%
Epoch [6/10], Training Loss: 0.976, Validation Accuracy: 58.98%
Epoch [7/10], Training Loss: 0.955, Validation Accuracy: 59.09%
Epoch [8/10], Training Loss: 0.953, Validation Accuracy: 59.00%
Epoch [9/10], Training Loss: 0.947, Validation Accuracy: 57.83%
Epoch [10/10], Training Loss: 0.932, Validation Accuracy: 58.77%
Epoch [1/10], Training Loss: 1.092, Validation Accuracy: 57.96%
Epoch [2/10], Training Loss: 1.050, Validation Accuracy: 59.08%
Epoch [3/10], Training Loss: 1.018, Validation Accuracy: 59.17%
Epoch [4/10], Training Loss: 1.003, Validation Accuracy: 59.10%
Epoch [5/10], Training Loss: 0.992, Validation Accuracy: 59.45%
Epoch [6/10], Training Loss: 0.970, Validation Accuracy: 58.52%
Epoch [7/10], Training Loss: 0.957, Validation Accuracy: 58.91%
Epoch [8/10], Training Loss: 0.948, Validation Accuracy: 59.06%
Epoch [9/10], Training Loss: 0.939, Validation Accuracy: 58.64%
Epoch [10/10], Training Loss: 0.925, Validation Accuracy: 59.15%
Epoch [1/10], Training Loss: 1.062, Validation Accuracy: 59.78%
Epoch [2/10], Training Loss: 1.023, Validation Accuracy: 59.45%
Epoch [3/10], Training Loss: 0.991, Validation Accuracy: 59.84%
Epoch [4/10], Training Loss: 0.977, Validation Accuracy: 58.81%
Epoch [5/10], Training Loss: 0.959, Validation Accuracy: 58.99%
Epoch [6/10], Training Loss: 0.936, Validation Accuracy: 59.27%
Epoch [7/10], Training Loss: 0.927, Validation Accuracy: 59.94%
Epoch [8/10], Training Loss: 0.913, Validation Accuracy: 59.54%
Epoch [9/10], Training Loss: 0.911, Validation Accuracy: 59.55%
Epoch [10/10], Training Loss: 0.889, Validation Accuracy: 58.33%
Epoch [1/10], Training Loss: 1.040, Validation Accuracy: 59.44%
Epoch [2/10], Training Loss: 1.000, Validation Accuracy: 59.81%
Epoch [3/10], Training Loss: 0.968, Validation Accuracy: 59.61%
Epoch [4/10], Training Loss: 0.946, Validation Accuracy: 58.84%
Epoch [5/10], Training Loss: 0.930, Validation Accuracy: 60.43%
Epoch [6/10], Training Loss: 0.912, Validation Accuracy: 59.06%
Epoch [7/10], Training Loss: 0.895, Validation Accuracy: 59.70%
Epoch [8/10], Training Loss: 0.881, Validation Accuracy: 60.03%
Epoch [9/10], Training Loss: 0.867, Validation Accuracy: 59.71%
Epoch [10/10], Training Loss: 0.865, Validation Accuracy: 59.29%
Epoch [1/10], Training Loss: 1.049, Validation Accuracy: 60.10%
Epoch [2/10], Training Loss: 0.998, Validation Accuracy: 60.12%
Epoch [3/10], Training Loss: 0.975, Validation Accuracy: 60.36%
Epoch [4/10], Training Loss: 0.943, Validation Accuracy: 59.86%
Epoch [5/10], Training Loss: 0.926, Validation Accuracy: 60.17%
Epoch [6/10], Training Loss: 0.913, Validation Accuracy: 59.83%
Epoch [7/10], Training Loss: 0.896, Validation Accuracy: 60.00%
Epoch [8/10], Training Loss: 0.886, Validation Accuracy: 59.51%
Epoch [9/10], Training Loss: 0.875, Validation Accuracy: 60.01%
Epoch [10/10], Training Loss: 0.852, Validation Accuracy: 59.38%
Epoch [1/10], Training Loss: 1.022, Validation Accuracy: 59.24%
Epoch [2/10], Training Loss: 0.966, Validation Accuracy: 59.76%
Epoch [3/10], Training Loss: 0.943, Validation Accuracy: 60.19%
Epoch [4/10], Training Loss: 0.913, Validation Accuracy: 60.48%
Epoch [5/10], Training Loss: 0.896, Validation Accuracy: 59.82%
Epoch [6/10], Training Loss: 0.887, Validation Accuracy: 60.22%
Epoch [7/10], Training Loss: 0.863, Validation Accuracy: 59.36%
Epoch [8/10], Training Loss: 0.850, Validation Accuracy: 59.22%
Epoch [9/10], Training Loss: 0.833, Validation Accuracy: 59.71%
Epoch [10/10], Training Loss: 0.826, Validation Accuracy: 58.98%
Epoch [1/10], Training Loss: 1.028, Validation Accuracy: 60.22%
Epoch [2/10], Training Loss: 0.974, Validation Accuracy: 59.12%
Epoch [3/10], Training Loss: 0.944, Validation Accuracy: 60.01%
Epoch [4/10], Training Loss: 0.927, Validation Accuracy: 59.59%
Epoch [5/10], Training Loss: 0.907, Validation Accuracy: 60.23%
Epoch [6/10], Training Loss: 0.883, Validation Accuracy: 60.00%
Epoch [7/10], Training Loss: 0.872, Validation Accuracy: 60.27%
Epoch [8/10], Training Loss: 0.864, Validation Accuracy: 60.26%
Epoch [9/10], Training Loss: 0.844, Validation Accuracy: 59.79%
Epoch [10/10], Training Loss: 0.826, Validation Accuracy: 59.36%
Epoch [1/10], Training Loss: 0.998, Validation Accuracy: 60.19%
Epoch [2/10], Training Loss: 0.946, Validation Accuracy: 60.71%
Epoch [3/10], Training Loss: 0.913, Validation Accuracy: 60.27%
Epoch [4/10], Training Loss: 0.893, Validation Accuracy: 60.66%
Epoch [5/10], Training Loss: 0.874, Validation Accuracy: 59.14%
Epoch [6/10], Training Loss: 0.852, Validation Accuracy: 60.96%
Epoch [7/10], Training Loss: 0.829, Validation Accuracy: 60.14%
Epoch [8/10], Training Loss: 0.807, Validation Accuracy: 60.43%
Epoch [9/10], Training Loss: 0.803, Validation Accuracy: 60.20%
Epoch [10/10], Training Loss: 0.788, Validation Accuracy: 60.82%
Epoch [1/10], Training Loss: 0.985, Validation Accuracy: 60.60%
Epoch [2/10], Training Loss: 0.927, Validation Accuracy: 61.42%
Epoch [3/10], Training Loss: 0.894, Validation Accuracy: 60.41%
Epoch [4/10], Training Loss: 0.867, Validation Accuracy: 60.87%
Epoch [5/10], Training Loss: 0.847, Validation Accuracy: 60.36%
Epoch [6/10], Training Loss: 0.835, Validation Accuracy: 60.32%
Epoch [7/10], Training Loss: 0.811, Validation Accuracy: 60.45%
Epoch [8/10], Training Loss: 0.792, Validation Accuracy: 60.25%
Epoch [9/10], Training Loss: 0.780, Validation Accuracy: 59.89%
Epoch [10/10], Training Loss: 0.762, Validation Accuracy: 60.73%
Epoch [1/10], Training Loss: 0.984, Validation Accuracy: 60.67%
Epoch [2/10], Training Loss: 0.922, Validation Accuracy: 60.58%
Epoch [3/10], Training Loss: 0.896, Validation Accuracy: 60.78%
Epoch [4/10], Training Loss: 0.868, Validation Accuracy: 60.89%
Epoch [5/10], Training Loss: 0.840, Validation Accuracy: 60.95%
Epoch [6/10], Training Loss: 0.817, Validation Accuracy: 60.69%
Epoch [7/10], Training Loss: 0.797, Validation Accuracy: 60.93%
Epoch [8/10], Training Loss: 0.779, Validation Accuracy: 60.70%
Epoch [9/10], Training Loss: 0.768, Validation Accuracy: 60.77%
Epoch [10/10], Training Loss: 0.751, Validation Accuracy: 61.07%
Epoch [1/10], Training Loss: 0.957, Validation Accuracy: 60.83%
Epoch [2/10], Training Loss: 0.899, Validation Accuracy: 61.02%
Epoch [3/10], Training Loss: 0.865, Validation Accuracy: 60.91%
Epoch [4/10], Training Loss: 0.835, Validation Accuracy: 60.02%
Epoch [5/10], Training Loss: 0.817, Validation Accuracy: 60.91%
Epoch [6/10], Training Loss: 0.802, Validation Accuracy: 60.70%
Epoch [7/10], Training Loss: 0.782, Validation Accuracy: 60.32%
Epoch [8/10], Training Loss: 0.770, Validation Accuracy: 60.83%
Epoch [9/10], Training Loss: 0.752, Validation Accuracy: 59.72%
Epoch [10/10], Training Loss: 0.733, Validation Accuracy: 60.95%
Epoch [1/10], Training Loss: 0.975, Validation Accuracy: 59.98%
Epoch [2/10], Training Loss: 0.908, Validation Accuracy: 60.47%
Epoch [3/10], Training Loss: 0.865, Validation Accuracy: 60.69%
Epoch [4/10], Training Loss: 0.837, Validation Accuracy: 60.36%
Epoch [5/10], Training Loss: 0.813, Validation Accuracy: 60.48%
Epoch [6/10], Training Loss: 0.798, Validation Accuracy: 61.11%
Epoch [7/10], Training Loss: 0.783, Validation Accuracy: 60.42%
Epoch [8/10], Training Loss: 0.765, Validation Accuracy: 60.92%
Epoch [9/10], Training Loss: 0.744, Validation Accuracy: 60.60%
Epoch [10/10], Training Loss: 0.733, Validation Accuracy: 59.79%
Epoch [1/10], Training Loss: 0.935, Validation Accuracy: 60.13%
Epoch [2/10], Training Loss: 0.875, Validation Accuracy: 60.84%
Epoch [3/10], Training Loss: 0.838, Validation Accuracy: 61.15%
Epoch [4/10], Training Loss: 0.811, Validation Accuracy: 61.19%
Epoch [5/10], Training Loss: 0.792, Validation Accuracy: 60.74%
Epoch [6/10], Training Loss: 0.764, Validation Accuracy: 59.01%
Epoch [7/10], Training Loss: 0.754, Validation Accuracy: 60.74%
Epoch [8/10], Training Loss: 0.728, Validation Accuracy: 61.23%
Epoch [9/10], Training Loss: 0.707, Validation Accuracy: 60.52%
Epoch [10/10], Training Loss: 0.701, Validation Accuracy: 60.61%
Epoch [1/10], Training Loss: 0.933, Validation Accuracy: 61.67%
Epoch [2/10], Training Loss: 0.853, Validation Accuracy: 61.20%
Epoch [3/10], Training Loss: 0.818, Validation Accuracy: 60.74%
Epoch [4/10], Training Loss: 0.794, Validation Accuracy: 61.38%
Epoch [5/10], Training Loss: 0.763, Validation Accuracy: 61.29%
Epoch [6/10], Training Loss: 0.735, Validation Accuracy: 61.16%
Epoch [7/10], Training Loss: 0.714, Validation Accuracy: 60.29%
Epoch [8/10], Training Loss: 0.702, Validation Accuracy: 60.64%
Epoch [9/10], Training Loss: 0.678, Validation Accuracy: 61.48%
Epoch [10/10], Training Loss: 0.668, Validation Accuracy: 60.80%
Epoch [1/10], Training Loss: 0.940, Validation Accuracy: 61.04%
Epoch [2/10], Training Loss: 0.855, Validation Accuracy: 61.06%
Epoch [3/10], Training Loss: 0.824, Validation Accuracy: 61.36%
Epoch [4/10], Training Loss: 0.778, Validation Accuracy: 61.31%
Epoch [5/10], Training Loss: 0.756, Validation Accuracy: 61.41%
Epoch [6/10], Training Loss: 0.740, Validation Accuracy: 61.24%
Epoch [7/10], Training Loss: 0.711, Validation Accuracy: 61.37%
Epoch [8/10], Training Loss: 0.687, Validation Accuracy: 61.25%
Epoch [9/10], Training Loss: 0.677, Validation Accuracy: 60.79%
Epoch [10/10], Training Loss: 0.663, Validation Accuracy: 60.72%
Epoch [1/10], Training Loss: 0.915, Validation Accuracy: 60.62%
Epoch [2/10], Training Loss: 0.831, Validation Accuracy: 61.22%
Epoch [3/10], Training Loss: 0.789, Validation Accuracy: 60.87%
Epoch [4/10], Training Loss: 0.755, Validation Accuracy: 61.13%
Epoch [5/10], Training Loss: 0.736, Validation Accuracy: 60.99%
Epoch [6/10], Training Loss: 0.713, Validation Accuracy: 60.44%
Epoch [7/10], Training Loss: 0.690, Validation Accuracy: 58.91%
Epoch [8/10], Training Loss: 0.674, Validation Accuracy: 60.34%
Epoch [9/10], Training Loss: 0.655, Validation Accuracy: 60.92%
Epoch [10/10], Training Loss: 0.635, Validation Accuracy: 60.49%
Epoch [1/10], Training Loss: 0.923, Validation Accuracy: 60.46%
Epoch [2/10], Training Loss: 0.848, Validation Accuracy: 59.97%
Epoch [3/10], Training Loss: 0.808, Validation Accuracy: 59.86%
Epoch [4/10], Training Loss: 0.764, Validation Accuracy: 60.63%
Epoch [5/10], Training Loss: 0.746, Validation Accuracy: 60.85%
Epoch [6/10], Training Loss: 0.719, Validation Accuracy: 61.18%
Epoch [7/10], Training Loss: 0.693, Validation Accuracy: 60.24%
Epoch [8/10], Training Loss: 0.677, Validation Accuracy: 60.66%
Epoch [9/10], Training Loss: 0.662, Validation Accuracy: 60.75%
Epoch [10/10], Training Loss: 0.649, Validation Accuracy: 60.20%
Epoch [1/10], Training Loss: 0.900, Validation Accuracy: 60.99%
Epoch [2/10], Training Loss: 0.822, Validation Accuracy: 61.16%
Epoch [3/10], Training Loss: 0.768, Validation Accuracy: 60.21%
Epoch [4/10], Training Loss: 0.731, Validation Accuracy: 61.58%
Epoch [5/10], Training Loss: 0.701, Validation Accuracy: 61.09%
Epoch [6/10], Training Loss: 0.678, Validation Accuracy: 61.55%
Epoch [7/10], Training Loss: 0.659, Validation Accuracy: 61.47%
Epoch [8/10], Training Loss: 0.633, Validation Accuracy: 60.93%
Epoch [9/10], Training Loss: 0.620, Validation Accuracy: 61.45%
Epoch [10/10], Training Loss: 0.602, Validation Accuracy: 61.18%
Epoch [1/10], Training Loss: 0.889, Validation Accuracy: 60.46%
Epoch [2/10], Training Loss: 0.798, Validation Accuracy: 61.45%
Epoch [3/10], Training Loss: 0.747, Validation Accuracy: 61.51%
Epoch [4/10], Training Loss: 0.719, Validation Accuracy: 61.59%
Epoch [5/10], Training Loss: 0.695, Validation Accuracy: 61.45%
Epoch [6/10], Training Loss: 0.652, Validation Accuracy: 61.19%
Epoch [7/10], Training Loss: 0.641, Validation Accuracy: 61.56%
Epoch [8/10], Training Loss: 0.610, Validation Accuracy: 61.38%
Epoch [9/10], Training Loss: 0.589, Validation Accuracy: 61.28%
Epoch [10/10], Training Loss: 0.577, Validation Accuracy: 60.69%
Epoch [1/10], Training Loss: 0.904, Validation Accuracy: 60.79%
Epoch [2/10], Training Loss: 0.790, Validation Accuracy: 61.49%
Epoch [3/10], Training Loss: 0.739, Validation Accuracy: 61.15%
Epoch [4/10], Training Loss: 0.701, Validation Accuracy: 61.60%
Epoch [5/10], Training Loss: 0.681, Validation Accuracy: 60.88%
Epoch [6/10], Training Loss: 0.646, Validation Accuracy: 61.32%
Epoch [7/10], Training Loss: 0.616, Validation Accuracy: 61.79%
Epoch [8/10], Training Loss: 0.606, Validation Accuracy: 61.34%
Epoch [9/10], Training Loss: 0.578, Validation Accuracy: 60.26%
Epoch [10/10], Training Loss: 0.566, Validation Accuracy: 61.27%
Epoch [1/10], Training Loss: 0.878, Validation Accuracy: 60.71%
Epoch [2/10], Training Loss: 0.790, Validation Accuracy: 61.10%
Epoch [3/10], Training Loss: 0.721, Validation Accuracy: 61.37%
Epoch [4/10], Training Loss: 0.680, Validation Accuracy: 60.79%
Epoch [5/10], Training Loss: 0.655, Validation Accuracy: 61.31%
Epoch [6/10], Training Loss: 0.624, Validation Accuracy: 61.27%
Epoch [7/10], Training Loss: 0.596, Validation Accuracy: 60.94%
Epoch [8/10], Training Loss: 0.575, Validation Accuracy: 60.72%
Epoch [9/10], Training Loss: 0.563, Validation Accuracy: 61.06%
Epoch [10/10], Training Loss: 0.540, Validation Accuracy: 60.40%
Epoch [1/10], Training Loss: 0.886, Validation Accuracy: 60.53%
Epoch [2/10], Training Loss: 0.786, Validation Accuracy: 60.56%
Epoch [3/10], Training Loss: 0.747, Validation Accuracy: 59.76%
Epoch [4/10], Training Loss: 0.704, Validation Accuracy: 60.99%
Epoch [5/10], Training Loss: 0.666, Validation Accuracy: 60.52%
Epoch [6/10], Training Loss: 0.641, Validation Accuracy: 60.46%
Epoch [7/10], Training Loss: 0.614, Validation Accuracy: 60.25%
Epoch [8/10], Training Loss: 0.597, Validation Accuracy: 60.14%
Epoch [9/10], Training Loss: 0.575, Validation Accuracy: 60.94%
Epoch [10/10], Training Loss: 0.552, Validation Accuracy: 59.90%
Epoch [1/10], Training Loss: 0.864, Validation Accuracy: 60.67%
Epoch [2/10], Training Loss: 0.753, Validation Accuracy: 61.56%
Epoch [3/10], Training Loss: 0.716, Validation Accuracy: 61.00%
Epoch [4/10], Training Loss: 0.667, Validation Accuracy: 61.21%
Epoch [5/10], Training Loss: 0.626, Validation Accuracy: 61.82%
Epoch [6/10], Training Loss: 0.605, Validation Accuracy: 61.73%
Epoch [7/10], Training Loss: 0.571, Validation Accuracy: 61.31%
Epoch [8/10], Training Loss: 0.559, Validation Accuracy: 60.66%
Epoch [9/10], Training Loss: 0.532, Validation Accuracy: 61.46%
Epoch [10/10], Training Loss: 0.514, Validation Accuracy: 60.34%
Epoch [1/10], Training Loss: 0.853, Validation Accuracy: 60.61%
Epoch [2/10], Training Loss: 0.742, Validation Accuracy: 61.47%
Epoch [3/10], Training Loss: 0.682, Validation Accuracy: 59.98%
Epoch [4/10], Training Loss: 0.643, Validation Accuracy: 60.94%
Epoch [5/10], Training Loss: 0.602, Validation Accuracy: 61.45%
Epoch [6/10], Training Loss: 0.580, Validation Accuracy: 61.44%
Epoch [7/10], Training Loss: 0.546, Validation Accuracy: 61.21%
Epoch [8/10], Training Loss: 0.531, Validation Accuracy: 60.77%
Epoch [9/10], Training Loss: 0.511, Validation Accuracy: 60.86%
Epoch [10/10], Training Loss: 0.498, Validation Accuracy: 60.70%
Epoch [1/10], Training Loss: 0.859, Validation Accuracy: 61.29%
Epoch [2/10], Training Loss: 0.737, Validation Accuracy: 60.20%
Epoch [3/10], Training Loss: 0.678, Validation Accuracy: 61.37%
Epoch [4/10], Training Loss: 0.631, Validation Accuracy: 61.41%
Epoch [5/10], Training Loss: 0.599, Validation Accuracy: 60.84%
Epoch [6/10], Training Loss: 0.571, Validation Accuracy: 61.21%
Epoch [7/10], Training Loss: 0.546, Validation Accuracy: 61.00%
Epoch [8/10], Training Loss: 0.514, Validation Accuracy: 60.84%
Epoch [9/10], Training Loss: 0.494, Validation Accuracy: 60.91%
Epoch [10/10], Training Loss: 0.499, Validation Accuracy: 61.12%
Epoch [1/10], Training Loss: 0.834, Validation Accuracy: 61.13%
Epoch [2/10], Training Loss: 0.712, Validation Accuracy: 61.00%
Epoch [3/10], Training Loss: 0.653, Validation Accuracy: 61.37%
Epoch [4/10], Training Loss: 0.610, Validation Accuracy: 61.69%
Epoch [5/10], Training Loss: 0.575, Validation Accuracy: 61.54%
Epoch [6/10], Training Loss: 0.548, Validation Accuracy: 60.88%
Epoch [7/10], Training Loss: 0.516, Validation Accuracy: 61.23%
Epoch [8/10], Training Loss: 0.494, Validation Accuracy: 60.62%
Epoch [9/10], Training Loss: 0.486, Validation Accuracy: 60.74%
Epoch [10/10], Training Loss: 0.464, Validation Accuracy: 60.85%
Epoch [1/10], Training Loss: 0.854, Validation Accuracy: 60.45%
Epoch [2/10], Training Loss: 0.731, Validation Accuracy: 60.59%
Epoch [3/10], Training Loss: 0.668, Validation Accuracy: 60.86%
Epoch [4/10], Training Loss: 0.624, Validation Accuracy: 60.20%
Epoch [5/10], Training Loss: 0.601, Validation Accuracy: 60.48%
Epoch [6/10], Training Loss: 0.558, Validation Accuracy: 60.94%
Epoch [7/10], Training Loss: 0.538, Validation Accuracy: 61.03%
Epoch [8/10], Training Loss: 0.507, Validation Accuracy: 60.52%
Epoch [9/10], Training Loss: 0.493, Validation Accuracy: 59.92%
Epoch [10/10], Training Loss: 0.471, Validation Accuracy: 60.18%
"""

# Regular expression to find validation accuracies
accuracies = re.findall(r'Validation Accuracy: (\d+\.\d+)%', log)

# Convert accuracies from string to float
accuracies = [float(acc) for acc in accuracies]

# Print accuracies
print("Accuracies:", accuracies)

# Print size of the array
print("Size of array:", len(accuracies))


Accuracies: [10.23, 10.39, 10.4, 10.46, 10.51, 11.06, 11.38, 12.02, 12.71, 14.52, 17.72, 17.84, 16.83, 17.6, 19.17, 20.78, 22.38, 23.42, 24.32, 26.29, 26.32, 26.74, 27.73, 28.39, 28.82, 29.19, 29.31, 29.99, 30.69, 31.64, 32.97, 33.05, 34.13, 34.44, 34.67, 35.97, 36.63, 36.75, 37.37, 38.02, 37.85, 39.17, 39.35, 39.82, 39.85, 40.52, 41.12, 40.65, 41.61, 42.0, 42.16, 41.07, 42.26, 42.12, 43.3, 43.31, 43.59, 44.72, 43.08, 44.2, 44.49, 44.97, 44.74, 45.18, 45.09, 45.35, 45.75, 45.42, 46.19, 46.37, 46.75, 46.4, 47.26, 46.66, 47.4, 47.26, 46.78, 47.24, 47.41, 47.62, 48.26, 47.29, 48.46, 47.92, 48.9, 48.59, 48.0, 48.68, 48.55, 48.74, 49.03, 49.57, 49.46, 49.47, 49.49, 50.03, 49.5, 49.78, 50.29, 50.01, 50.54, 49.81, 50.71, 51.03, 51.24, 50.52, 51.22, 51.07, 51.01, 50.84, 51.86, 51.89, 51.9, 52.18, 51.86, 52.5, 52.05, 50.8, 52.27, 52.33, 52.94, 52.61, 52.37, 52.04, 52.91, 52.78, 52.36, 53.43, 53.43, 53.06, 53.18, 53.77, 53.35, 53.48, 53.38, 53.05, 53.4, 53.66, 53.54, 54.2, 53.92, 53.26, 54.0, 54

In [ ]:
# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 10  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=5)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()

Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 10.54%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 10.89%
Epoch [3/10], Training Loss: 2.305, Validation Accuracy: 10.83%
Epoch [4/10], Training Loss: 2.304, Validation Accuracy: 11.51%
Epoch [5/10], Training Loss: 2.304, Validation Accuracy: 11.62%
Epoch [6/10], Training Loss: 2.303, Validation Accuracy: 12.56%
Epoch [7/10], Training Loss: 2.303, Validation Accuracy: 12.60%
Epoch [8/10], Training Loss: 2.302, Validation Accuracy: 13.07%
Epoch [9/10], Training Loss: 2.301, Validation Accuracy: 13.95%
Epoch [10/10], Training Loss: 2.300, Validation Accuracy: 13.66%
Epoch [1/10], Training Loss: 2.300, Validation Accuracy: 13.22%
Epoch [2/10], Training Loss: 2.300, Validation Accuracy: 12.61%
Epoch [3/10], Training Loss: 2.299, Validation Accuracy: 12.54%
Epoch [4/10], Training Loss: 2.299, Validation Accuracy: 12.38%
Epoch [5/10], Training Loss: 2.298, Validation Accuracy: 12.53%
Epoch [6/10], Training Loss: 2.297, Val

In [ ]:
import re

# Your provided text
log = """
Epoch [1/10], Training Loss: 2.305, Validation Accuracy: 10.04%
Epoch [2/10], Training Loss: 2.304, Validation Accuracy: 10.78%
Epoch [3/10], Training Loss: 2.303, Validation Accuracy: 11.40%
Epoch [4/10], Training Loss: 2.302, Validation Accuracy: 11.70%
Epoch [5/10], Training Loss: 2.301, Validation Accuracy: 11.99%
Epoch [6/10], Training Loss: 2.300, Validation Accuracy: 11.82%
Epoch [7/10], Training Loss: 2.299, Validation Accuracy: 11.51%
Epoch [8/10], Training Loss: 2.298, Validation Accuracy: 11.33%
Epoch [9/10], Training Loss: 2.296, Validation Accuracy: 11.38%
Epoch [10/10], Training Loss: 2.295, Validation Accuracy: 11.46%
Epoch [1/10], Training Loss: 2.293, Validation Accuracy: 11.43%
Epoch [2/10], Training Loss: 2.290, Validation Accuracy: 11.86%
Epoch [3/10], Training Loss: 2.287, Validation Accuracy: 12.51%
Epoch [4/10], Training Loss: 2.282, Validation Accuracy: 13.75%
Epoch [5/10], Training Loss: 2.274, Validation Accuracy: 14.94%
Epoch [6/10], Training Loss: 2.263, Validation Accuracy: 16.29%
Epoch [7/10], Training Loss: 2.246, Validation Accuracy: 17.40%
Epoch [8/10], Training Loss: 2.220, Validation Accuracy: 18.76%
Epoch [9/10], Training Loss: 2.190, Validation Accuracy: 20.65%
Epoch [10/10], Training Loss: 2.162, Validation Accuracy: 23.27%
Epoch [1/10], Training Loss: 2.139, Validation Accuracy: 24.55%
Epoch [2/10], Training Loss: 2.121, Validation Accuracy: 25.06%
Epoch [3/10], Training Loss: 2.104, Validation Accuracy: 25.58%
Epoch [4/10], Training Loss: 2.091, Validation Accuracy: 25.80%
Epoch [5/10], Training Loss: 2.076, Validation Accuracy: 26.40%
Epoch [6/10], Training Loss: 2.061, Validation Accuracy: 27.51%
Epoch [7/10], Training Loss: 2.045, Validation Accuracy: 28.46%
Epoch [8/10], Training Loss: 2.026, Validation Accuracy: 28.40%
Epoch [9/10], Training Loss: 2.008, Validation Accuracy: 28.93%
Epoch [10/10], Training Loss: 1.982, Validation Accuracy: 30.03%
Epoch [1/10], Training Loss: 1.968, Validation Accuracy: 31.17%
Epoch [2/10], Training Loss: 1.941, Validation Accuracy: 31.15%
Epoch [3/10], Training Loss: 1.909, Validation Accuracy: 31.97%
Epoch [4/10], Training Loss: 1.885, Validation Accuracy: 32.09%
Epoch [5/10], Training Loss: 1.865, Validation Accuracy: 32.10%
Epoch [6/10], Training Loss: 1.847, Validation Accuracy: 33.37%
Epoch [7/10], Training Loss: 1.826, Validation Accuracy: 33.20%
Epoch [8/10], Training Loss: 1.803, Validation Accuracy: 34.15%
Epoch [9/10], Training Loss: 1.786, Validation Accuracy: 34.71%
Epoch [10/10], Training Loss: 1.769, Validation Accuracy: 35.26%
Epoch [1/10], Training Loss: 1.777, Validation Accuracy: 35.33%
Epoch [2/10], Training Loss: 1.756, Validation Accuracy: 36.05%
Epoch [3/10], Training Loss: 1.742, Validation Accuracy: 35.13%
Epoch [4/10], Training Loss: 1.735, Validation Accuracy: 36.08%
Epoch [5/10], Training Loss: 1.717, Validation Accuracy: 36.72%
Epoch [6/10], Training Loss: 1.707, Validation Accuracy: 37.83%
Epoch [7/10], Training Loss: 1.683, Validation Accuracy: 37.47%
Epoch [8/10], Training Loss: 1.677, Validation Accuracy: 37.17%
Epoch [9/10], Training Loss: 1.664, Validation Accuracy: 38.33%
Epoch [10/10], Training Loss: 1.654, Validation Accuracy: 39.32%
Epoch [1/10], Training Loss: 1.677, Validation Accuracy: 39.08%
Epoch [2/10], Training Loss: 1.660, Validation Accuracy: 39.65%
Epoch [3/10], Training Loss: 1.645, Validation Accuracy: 39.70%
Epoch [4/10], Training Loss: 1.633, Validation Accuracy: 40.42%
Epoch [5/10], Training Loss: 1.624, Validation Accuracy: 40.39%
Epoch [6/10], Training Loss: 1.613, Validation Accuracy: 40.50%
Epoch [7/10], Training Loss: 1.603, Validation Accuracy: 40.47%
Epoch [8/10], Training Loss: 1.593, Validation Accuracy: 41.93%
Epoch [9/10], Training Loss: 1.579, Validation Accuracy: 41.17%
Epoch [10/10], Training Loss: 1.575, Validation Accuracy: 41.28%
Epoch [1/10], Training Loss: 1.633, Validation Accuracy: 41.96%
Epoch [2/10], Training Loss: 1.615, Validation Accuracy: 42.78%
Epoch [3/10], Training Loss: 1.605, Validation Accuracy: 42.13%
Epoch [4/10], Training Loss: 1.594, Validation Accuracy: 42.36%
Epoch [5/10], Training Loss: 1.586, Validation Accuracy: 42.97%
Epoch [6/10], Training Loss: 1.573, Validation Accuracy: 43.05%
Epoch [7/10], Training Loss: 1.563, Validation Accuracy: 43.32%
Epoch [8/10], Training Loss: 1.553, Validation Accuracy: 43.55%
Epoch [9/10], Training Loss: 1.543, Validation Accuracy: 44.07%
Epoch [10/10], Training Loss: 1.532, Validation Accuracy: 44.25%
Epoch [1/10], Training Loss: 1.520, Validation Accuracy: 43.37%
Epoch [2/10], Training Loss: 1.508, Validation Accuracy: 44.78%
Epoch [3/10], Training Loss: 1.489, Validation Accuracy: 44.92%
Epoch [4/10], Training Loss: 1.486, Validation Accuracy: 44.52%
Epoch [5/10], Training Loss: 1.477, Validation Accuracy: 45.19%
Epoch [6/10], Training Loss: 1.464, Validation Accuracy: 45.20%
Epoch [7/10], Training Loss: 1.452, Validation Accuracy: 45.03%
Epoch [8/10], Training Loss: 1.451, Validation Accuracy: 46.05%
Epoch [9/10], Training Loss: 1.430, Validation Accuracy: 45.85%
Epoch [10/10], Training Loss: 1.425, Validation Accuracy: 46.56%
Epoch [1/10], Training Loss: 1.451, Validation Accuracy: 46.45%
Epoch [2/10], Training Loss: 1.440, Validation Accuracy: 46.63%
Epoch [3/10], Training Loss: 1.431, Validation Accuracy: 47.03%
Epoch [4/10], Training Loss: 1.417, Validation Accuracy: 47.31%
Epoch [5/10], Training Loss: 1.402, Validation Accuracy: 46.22%
Epoch [6/10], Training Loss: 1.403, Validation Accuracy: 47.19%
Epoch [7/10], Training Loss: 1.397, Validation Accuracy: 47.72%
Epoch [8/10], Training Loss: 1.379, Validation Accuracy: 48.12%
Epoch [9/10], Training Loss: 1.378, Validation Accuracy: 47.44%
Epoch [10/10], Training Loss: 1.365, Validation Accuracy: 48.09%
Epoch [1/10], Training Loss: 1.422, Validation Accuracy: 48.06%
Epoch [2/10], Training Loss: 1.410, Validation Accuracy: 48.26%
Epoch [3/10], Training Loss: 1.391, Validation Accuracy: 48.70%
Epoch [4/10], Training Loss: 1.384, Validation Accuracy: 48.05%
Epoch [5/10], Training Loss: 1.374, Validation Accuracy: 48.69%
Epoch [6/10], Training Loss: 1.363, Validation Accuracy: 49.05%
Epoch [7/10], Training Loss: 1.359, Validation Accuracy: 47.70%
Epoch [8/10], Training Loss: 1.361, Validation Accuracy: 48.53%
Epoch [9/10], Training Loss: 1.344, Validation Accuracy: 49.62%
Epoch [10/10], Training Loss: 1.335, Validation Accuracy: 49.59%
Epoch [1/10], Training Loss: 1.395, Validation Accuracy: 49.24%
Epoch [2/10], Training Loss: 1.376, Validation Accuracy: 49.81%
Epoch [3/10], Training Loss: 1.359, Validation Accuracy: 50.83%
Epoch [4/10], Training Loss: 1.351, Validation Accuracy: 49.15%
Epoch [5/10], Training Loss: 1.338, Validation Accuracy: 51.02%
Epoch [6/10], Training Loss: 1.334, Validation Accuracy: 49.88%
Epoch [7/10], Training Loss: 1.328, Validation Accuracy: 50.12%
Epoch [8/10], Training Loss: 1.319, Validation Accuracy: 50.60%
Epoch [9/10], Training Loss: 1.308, Validation Accuracy: 49.96%
Epoch [10/10], Training Loss: 1.304, Validation Accuracy: 51.31%
Epoch [1/10], Training Loss: 1.381, Validation Accuracy: 50.29%
Epoch [2/10], Training Loss: 1.361, Validation Accuracy: 51.65%
Epoch [3/10], Training Loss: 1.349, Validation Accuracy: 50.89%
Epoch [4/10], Training Loss: 1.335, Validation Accuracy: 50.70%
Epoch [5/10], Training Loss: 1.333, Validation Accuracy: 52.40%
Epoch [6/10], Training Loss: 1.322, Validation Accuracy: 51.84%
Epoch [7/10], Training Loss: 1.319, Validation Accuracy: 51.16%
Epoch [8/10], Training Loss: 1.307, Validation Accuracy: 51.69%
Epoch [9/10], Training Loss: 1.292, Validation Accuracy: 52.57%
Epoch [10/10], Training Loss: 1.282, Validation Accuracy: 52.05%
Epoch [1/10], Training Loss: 1.320, Validation Accuracy: 52.26%
Epoch [2/10], Training Loss: 1.294, Validation Accuracy: 51.97%
Epoch [3/10], Training Loss: 1.282, Validation Accuracy: 52.06%
Epoch [4/10], Training Loss: 1.267, Validation Accuracy: 52.88%
Epoch [5/10], Training Loss: 1.252, Validation Accuracy: 52.62%
Epoch [6/10], Training Loss: 1.251, Validation Accuracy: 52.25%
Epoch [7/10], Training Loss: 1.242, Validation Accuracy: 53.48%
Epoch [8/10], Training Loss: 1.228, Validation Accuracy: 52.86%
Epoch [9/10], Training Loss: 1.222, Validation Accuracy: 51.74%
Epoch [10/10], Training Loss: 1.218, Validation Accuracy: 52.79%
Epoch [1/10], Training Loss: 1.284, Validation Accuracy: 53.37%
Epoch [2/10], Training Loss: 1.257, Validation Accuracy: 53.38%
Epoch [3/10], Training Loss: 1.244, Validation Accuracy: 53.78%
Epoch [4/10], Training Loss: 1.235, Validation Accuracy: 53.45%
Epoch [5/10], Training Loss: 1.223, Validation Accuracy: 54.44%
Epoch [6/10], Training Loss: 1.216, Validation Accuracy: 53.64%
Epoch [7/10], Training Loss: 1.203, Validation Accuracy: 54.34%
Epoch [8/10], Training Loss: 1.194, Validation Accuracy: 54.23%
Epoch [9/10], Training Loss: 1.197, Validation Accuracy: 52.83%
Epoch [10/10], Training Loss: 1.179, Validation Accuracy: 54.44%
Epoch [1/10], Training Loss: 1.265, Validation Accuracy: 51.78%
Epoch [2/10], Training Loss: 1.273, Validation Accuracy: 54.93%
Epoch [3/10], Training Loss: 1.237, Validation Accuracy: 54.30%
Epoch [4/10], Training Loss: 1.219, Validation Accuracy: 53.76%
Epoch [5/10], Training Loss: 1.198, Validation Accuracy: 54.73%
Epoch [6/10], Training Loss: 1.199, Validation Accuracy: 55.49%
Epoch [7/10], Training Loss: 1.187, Validation Accuracy: 54.89%
Epoch [8/10], Training Loss: 1.180, Validation Accuracy: 55.41%
Epoch [9/10], Training Loss: 1.167, Validation Accuracy: 54.78%
Epoch [10/10], Training Loss: 1.163, Validation Accuracy: 55.71%
Epoch [1/10], Training Loss: 1.258, Validation Accuracy: 55.33%
Epoch [2/10], Training Loss: 1.227, Validation Accuracy: 55.27%
Epoch [3/10], Training Loss: 1.203, Validation Accuracy: 54.99%
Epoch [4/10], Training Loss: 1.188, Validation Accuracy: 55.41%
Epoch [5/10], Training Loss: 1.179, Validation Accuracy: 55.40%
Epoch [6/10], Training Loss: 1.172, Validation Accuracy: 55.70%
Epoch [7/10], Training Loss: 1.160, Validation Accuracy: 55.87%
Epoch [8/10], Training Loss: 1.145, Validation Accuracy: 54.13%
Epoch [9/10], Training Loss: 1.138, Validation Accuracy: 55.39%
Epoch [10/10], Training Loss: 1.125, Validation Accuracy: 55.85%
Epoch [1/10], Training Loss: 1.239, Validation Accuracy: 56.05%
Epoch [2/10], Training Loss: 1.210, Validation Accuracy: 55.29%
Epoch [3/10], Training Loss: 1.201, Validation Accuracy: 55.98%
Epoch [4/10], Training Loss: 1.188, Validation Accuracy: 55.84%
Epoch [5/10], Training Loss: 1.176, Validation Accuracy: 56.36%
Epoch [6/10], Training Loss: 1.155, Validation Accuracy: 56.28%
Epoch [7/10], Training Loss: 1.152, Validation Accuracy: 56.18%
Epoch [8/10], Training Loss: 1.145, Validation Accuracy: 55.30%
Epoch [9/10], Training Loss: 1.142, Validation Accuracy: 54.20%
Epoch [10/10], Training Loss: 1.122, Validation Accuracy: 56.52%
Epoch [1/10], Training Loss: 1.191, Validation Accuracy: 56.84%
Epoch [2/10], Training Loss: 1.172, Validation Accuracy: 55.99%
Epoch [3/10], Training Loss: 1.146, Validation Accuracy: 56.09%
Epoch [4/10], Training Loss: 1.147, Validation Accuracy: 57.10%
Epoch [5/10], Training Loss: 1.123, Validation Accuracy: 57.28%
Epoch [6/10], Training Loss: 1.111, Validation Accuracy: 56.85%
Epoch [7/10], Training Loss: 1.102, Validation Accuracy: 56.53%
Epoch [8/10], Training Loss: 1.093, Validation Accuracy: 56.86%
Epoch [9/10], Training Loss: 1.083, Validation Accuracy: 56.50%
Epoch [10/10], Training Loss: 1.074, Validation Accuracy: 57.02%
Epoch [1/10], Training Loss: 1.171, Validation Accuracy: 56.97%
Epoch [2/10], Training Loss: 1.140, Validation Accuracy: 56.81%
Epoch [3/10], Training Loss: 1.119, Validation Accuracy: 56.91%
Epoch [4/10], Training Loss: 1.102, Validation Accuracy: 56.39%
Epoch [5/10], Training Loss: 1.096, Validation Accuracy: 56.59%
Epoch [6/10], Training Loss: 1.084, Validation Accuracy: 57.20%
Epoch [7/10], Training Loss: 1.069, Validation Accuracy: 57.44%
Epoch [8/10], Training Loss: 1.060, Validation Accuracy: 57.69%
Epoch [9/10], Training Loss: 1.051, Validation Accuracy: 57.37%
Epoch [10/10], Training Loss: 1.037, Validation Accuracy: 57.09%
Epoch [1/10], Training Loss: 1.165, Validation Accuracy: 58.24%
Epoch [2/10], Training Loss: 1.130, Validation Accuracy: 57.43%
Epoch [3/10], Training Loss: 1.107, Validation Accuracy: 58.08%
Epoch [4/10], Training Loss: 1.091, Validation Accuracy: 58.14%
Epoch [5/10], Training Loss: 1.078, Validation Accuracy: 58.06%
Epoch [6/10], Training Loss: 1.063, Validation Accuracy: 57.68%
Epoch [7/10], Training Loss: 1.061, Validation Accuracy: 58.31%
Epoch [8/10], Training Loss: 1.045, Validation Accuracy: 58.51%
Epoch [9/10], Training Loss: 1.034, Validation Accuracy: 57.50%
Epoch [10/10], Training Loss: 1.019, Validation Accuracy: 56.47%
Epoch [1/10], Training Loss: 1.140, Validation Accuracy: 57.72%
Epoch [2/10], Training Loss: 1.101, Validation Accuracy: 58.52%
Epoch [3/10], Training Loss: 1.081, Validation Accuracy: 58.36%
Epoch [4/10], Training Loss: 1.069, Validation Accuracy: 57.61%
Epoch [5/10], Training Loss: 1.063, Validation Accuracy: 58.42%
Epoch [6/10], Training Loss: 1.039, Validation Accuracy: 56.58%
Epoch [7/10], Training Loss: 1.034, Validation Accuracy: 58.41%
Epoch [8/10], Training Loss: 1.022, Validation Accuracy: 58.10%
Epoch [9/10], Training Loss: 1.005, Validation Accuracy: 57.99%
Epoch [10/10], Training Loss: 0.999, Validation Accuracy: 57.56%
Epoch [1/10], Training Loss: 1.149, Validation Accuracy: 58.03%
Epoch [2/10], Training Loss: 1.113, Validation Accuracy: 59.04%
Epoch [3/10], Training Loss: 1.081, Validation Accuracy: 58.37%
Epoch [4/10], Training Loss: 1.067, Validation Accuracy: 58.63%
Epoch [5/10], Training Loss: 1.048, Validation Accuracy: 58.73%
Epoch [6/10], Training Loss: 1.042, Validation Accuracy: 58.43%
Epoch [7/10], Training Loss: 1.032, Validation Accuracy: 58.15%
Epoch [8/10], Training Loss: 1.025, Validation Accuracy: 57.82%
Epoch [9/10], Training Loss: 1.003, Validation Accuracy: 58.63%
Epoch [10/10], Training Loss: 0.998, Validation Accuracy: 58.36%
Epoch [1/10], Training Loss: 1.101, Validation Accuracy: 58.56%
Epoch [2/10], Training Loss: 1.060, Validation Accuracy: 58.40%
Epoch [3/10], Training Loss: 1.041, Validation Accuracy: 59.53%
Epoch [4/10], Training Loss: 1.015, Validation Accuracy: 59.06%
Epoch [5/10], Training Loss: 1.004, Validation Accuracy: 58.58%
Epoch [6/10], Training Loss: 0.991, Validation Accuracy: 58.32%
Epoch [7/10], Training Loss: 0.980, Validation Accuracy: 59.33%
Epoch [8/10], Training Loss: 0.960, Validation Accuracy: 59.55%
Epoch [9/10], Training Loss: 0.951, Validation Accuracy: 58.60%
Epoch [10/10], Training Loss: 0.954, Validation Accuracy: 57.83%
Epoch [1/10], Training Loss: 1.081, Validation Accuracy: 59.57%
Epoch [2/10], Training Loss: 1.039, Validation Accuracy: 59.29%
Epoch [3/10], Training Loss: 1.016, Validation Accuracy: 59.27%
Epoch [4/10], Training Loss: 1.001, Validation Accuracy: 59.18%
Epoch [5/10], Training Loss: 0.992, Validation Accuracy: 59.24%
Epoch [6/10], Training Loss: 0.970, Validation Accuracy: 58.64%
Epoch [7/10], Training Loss: 0.961, Validation Accuracy: 59.05%
Epoch [8/10], Training Loss: 0.943, Validation Accuracy: 59.76%
Epoch [9/10], Training Loss: 0.937, Validation Accuracy: 58.58%
Epoch [10/10], Training Loss: 0.918, Validation Accuracy: 59.51%
Epoch [1/10], Training Loss: 1.072, Validation Accuracy: 59.68%
Epoch [2/10], Training Loss: 1.050, Validation Accuracy: 59.50%
Epoch [3/10], Training Loss: 1.008, Validation Accuracy: 59.27%
Epoch [4/10], Training Loss: 0.989, Validation Accuracy: 59.77%
Epoch [5/10], Training Loss: 0.972, Validation Accuracy: 59.26%
Epoch [6/10], Training Loss: 0.964, Validation Accuracy: 59.13%
Epoch [7/10], Training Loss: 0.953, Validation Accuracy: 59.33%
Epoch [8/10], Training Loss: 0.930, Validation Accuracy: 58.98%
Epoch [9/10], Training Loss: 0.912, Validation Accuracy: 58.62%
Epoch [10/10], Training Loss: 0.907, Validation Accuracy: 59.56%
Epoch [1/10], Training Loss: 1.066, Validation Accuracy: 59.30%
Epoch [2/10], Training Loss: 1.026, Validation Accuracy: 58.82%
Epoch [3/10], Training Loss: 1.001, Validation Accuracy: 58.98%
Epoch [4/10], Training Loss: 0.979, Validation Accuracy: 59.54%
Epoch [5/10], Training Loss: 0.959, Validation Accuracy: 59.23%
Epoch [6/10], Training Loss: 0.939, Validation Accuracy: 60.37%
Epoch [7/10], Training Loss: 0.934, Validation Accuracy: 60.26%
Epoch [8/10], Training Loss: 0.914, Validation Accuracy: 59.53%
Epoch [9/10], Training Loss: 0.894, Validation Accuracy: 59.57%
Epoch [10/10], Training Loss: 0.882, Validation Accuracy: 59.44%
Epoch [1/10], Training Loss: 1.075, Validation Accuracy: 59.70%
Epoch [2/10], Training Loss: 1.037, Validation Accuracy: 59.92%
Epoch [3/10], Training Loss: 1.001, Validation Accuracy: 59.77%
Epoch [4/10], Training Loss: 0.968, Validation Accuracy: 60.50%
Epoch [5/10], Training Loss: 0.949, Validation Accuracy: 59.80%
Epoch [6/10], Training Loss: 0.942, Validation Accuracy: 59.79%
Epoch [7/10], Training Loss: 0.923, Validation Accuracy: 59.80%
Epoch [8/10], Training Loss: 0.905, Validation Accuracy: 60.09%
Epoch [9/10], Training Loss: 0.892, Validation Accuracy: 60.32%
Epoch [10/10], Training Loss: 0.881, Validation Accuracy: 59.73%
Epoch [1/10], Training Loss: 1.030, Validation Accuracy: 60.64%
Epoch [2/10], Training Loss: 0.970, Validation Accuracy: 60.32%
Epoch [3/10], Training Loss: 0.955, Validation Accuracy: 60.15%
Epoch [4/10], Training Loss: 0.929, Validation Accuracy: 60.26%
Epoch [5/10], Training Loss: 0.904, Validation Accuracy: 60.22%
Epoch [6/10], Training Loss: 0.896, Validation Accuracy: 60.01%
Epoch [7/10], Training Loss: 0.873, Validation Accuracy: 60.41%
Epoch [8/10], Training Loss: 0.857, Validation Accuracy: 60.85%
Epoch [9/10], Training Loss: 0.849, Validation Accuracy: 59.96%
Epoch [10/10], Training Loss: 0.839, Validation Accuracy: 60.41%
Epoch [1/10], Training Loss: 1.013, Validation Accuracy: 60.37%
Epoch [2/10], Training Loss: 0.968, Validation Accuracy: 60.05%
Epoch [3/10], Training Loss: 0.937, Validation Accuracy: 59.81%
Epoch [4/10], Training Loss: 0.910, Validation Accuracy: 60.43%
Epoch [5/10], Training Loss: 0.885, Validation Accuracy: 60.35%
Epoch [6/10], Training Loss: 0.873, Validation Accuracy: 60.23%
Epoch [7/10], Training Loss: 0.861, Validation Accuracy: 60.41%
Epoch [8/10], Training Loss: 0.840, Validation Accuracy: 59.96%
Epoch [9/10], Training Loss: 0.836, Validation Accuracy: 60.56%
Epoch [10/10], Training Loss: 0.819, Validation Accuracy: 60.56%
Epoch [1/10], Training Loss: 1.025, Validation Accuracy: 60.81%
Epoch [2/10], Training Loss: 0.964, Validation Accuracy: 60.77%
Epoch [3/10], Training Loss: 0.928, Validation Accuracy: 60.66%
Epoch [4/10], Training Loss: 0.905, Validation Accuracy: 60.73%
Epoch [5/10], Training Loss: 0.878, Validation Accuracy: 60.01%
Epoch [6/10], Training Loss: 0.864, Validation Accuracy: 60.30%
Epoch [7/10], Training Loss: 0.855, Validation Accuracy: 60.22%
Epoch [8/10], Training Loss: 0.838, Validation Accuracy: 60.89%
Epoch [9/10], Training Loss: 0.809, Validation Accuracy: 59.89%
Epoch [10/10], Training Loss: 0.807, Validation Accuracy: 60.03%
Epoch [1/10], Training Loss: 0.995, Validation Accuracy: 60.17%
Epoch [2/10], Training Loss: 0.940, Validation Accuracy: 60.63%
Epoch [3/10], Training Loss: 0.909, Validation Accuracy: 60.33%
Epoch [4/10], Training Loss: 0.886, Validation Accuracy: 60.29%
Epoch [5/10], Training Loss: 0.869, Validation Accuracy: 60.36%
Epoch [6/10], Training Loss: 0.848, Validation Accuracy: 60.91%
Epoch [7/10], Training Loss: 0.831, Validation Accuracy: 60.14%
Epoch [8/10], Training Loss: 0.821, Validation Accuracy: 59.25%
Epoch [9/10], Training Loss: 0.810, Validation Accuracy: 60.33%
Epoch [10/10], Training Loss: 0.789, Validation Accuracy: 60.37%
Epoch [1/10], Training Loss: 1.001, Validation Accuracy: 59.96%
Epoch [2/10], Training Loss: 0.952, Validation Accuracy: 60.87%
Epoch [3/10], Training Loss: 0.906, Validation Accuracy: 61.25%
Epoch [4/10], Training Loss: 0.886, Validation Accuracy: 60.88%
Epoch [5/10], Training Loss: 0.862, Validation Accuracy: 61.25%
Epoch [6/10], Training Loss: 0.845, Validation Accuracy: 60.72%
Epoch [7/10], Training Loss: 0.823, Validation Accuracy: 61.17%
Epoch [8/10], Training Loss: 0.804, Validation Accuracy: 60.38%
Epoch [9/10], Training Loss: 0.787, Validation Accuracy: 61.07%
Epoch [10/10], Training Loss: 0.780, Validation Accuracy: 59.82%
Epoch [1/10], Training Loss: 0.971, Validation Accuracy: 61.02%
Epoch [2/10], Training Loss: 0.911, Validation Accuracy: 61.31%
Epoch [3/10], Training Loss: 0.863, Validation Accuracy: 61.01%
Epoch [4/10], Training Loss: 0.844, Validation Accuracy: 61.45%
Epoch [5/10], Training Loss: 0.821, Validation Accuracy: 60.65%
Epoch [6/10], Training Loss: 0.803, Validation Accuracy: 61.44%
Epoch [7/10], Training Loss: 0.783, Validation Accuracy: 60.96%
Epoch [8/10], Training Loss: 0.777, Validation Accuracy: 60.14%
Epoch [9/10], Training Loss: 0.754, Validation Accuracy: 61.10%
Epoch [10/10], Training Loss: 0.740, Validation Accuracy: 61.37%
Epoch [1/10], Training Loss: 0.944, Validation Accuracy: 61.03%
Epoch [2/10], Training Loss: 0.896, Validation Accuracy: 61.48%
Epoch [3/10], Training Loss: 0.873, Validation Accuracy: 61.20%
Epoch [4/10], Training Loss: 0.825, Validation Accuracy: 61.31%
Epoch [5/10], Training Loss: 0.803, Validation Accuracy: 60.95%
Epoch [6/10], Training Loss: 0.789, Validation Accuracy: 60.76%
Epoch [7/10], Training Loss: 0.766, Validation Accuracy: 60.87%
Epoch [8/10], Training Loss: 0.753, Validation Accuracy: 60.84%
Epoch [9/10], Training Loss: 0.736, Validation Accuracy: 60.54%
Epoch [10/10], Training Loss: 0.718, Validation Accuracy: 60.23%
Epoch [1/10], Training Loss: 0.953, Validation Accuracy: 61.74%
Epoch [2/10], Training Loss: 0.888, Validation Accuracy: 60.70%
Epoch [3/10], Training Loss: 0.850, Validation Accuracy: 60.87%
Epoch [4/10], Training Loss: 0.825, Validation Accuracy: 60.93%
Epoch [5/10], Training Loss: 0.802, Validation Accuracy: 60.75%
Epoch [6/10], Training Loss: 0.777, Validation Accuracy: 61.52%
Epoch [7/10], Training Loss: 0.747, Validation Accuracy: 61.02%
Epoch [8/10], Training Loss: 0.745, Validation Accuracy: 60.71%
Epoch [9/10], Training Loss: 0.731, Validation Accuracy: 60.33%
Epoch [10/10], Training Loss: 0.712, Validation Accuracy: 60.55%
Epoch [1/10], Training Loss: 0.943, Validation Accuracy: 61.59%
Epoch [2/10], Training Loss: 0.876, Validation Accuracy: 60.98%
Epoch [3/10], Training Loss: 0.830, Validation Accuracy: 61.25%
Epoch [4/10], Training Loss: 0.816, Validation Accuracy: 61.34%
Epoch [5/10], Training Loss: 0.782, Validation Accuracy: 61.59%
Epoch [6/10], Training Loss: 0.758, Validation Accuracy: 60.38%
Epoch [7/10], Training Loss: 0.753, Validation Accuracy: 60.43%
Epoch [8/10], Training Loss: 0.728, Validation Accuracy: 60.19%
Epoch [9/10], Training Loss: 0.712, Validation Accuracy: 60.96%
Epoch [10/10], Training Loss: 0.692, Validation Accuracy: 60.61%
Epoch [1/10], Training Loss: 0.945, Validation Accuracy: 61.08%
Epoch [2/10], Training Loss: 0.880, Validation Accuracy: 61.35%
Epoch [3/10], Training Loss: 0.836, Validation Accuracy: 61.97%
Epoch [4/10], Training Loss: 0.803, Validation Accuracy: 61.84%
Epoch [5/10], Training Loss: 0.776, Validation Accuracy: 61.76%
Epoch [6/10], Training Loss: 0.752, Validation Accuracy: 61.48%
Epoch [7/10], Training Loss: 0.731, Validation Accuracy: 61.10%
Epoch [8/10], Training Loss: 0.709, Validation Accuracy: 61.44%
Epoch [9/10], Training Loss: 0.696, Validation Accuracy: 60.94%
Epoch [10/10], Training Loss: 0.680, Validation Accuracy: 61.01%
Epoch [1/10], Training Loss: 0.919, Validation Accuracy: 61.01%
Epoch [2/10], Training Loss: 0.835, Validation Accuracy: 61.51%
Epoch [3/10], Training Loss: 0.822, Validation Accuracy: 60.92%
Epoch [4/10], Training Loss: 0.772, Validation Accuracy: 60.84%
Epoch [5/10], Training Loss: 0.739, Validation Accuracy: 61.77%
Epoch [6/10], Training Loss: 0.711, Validation Accuracy: 61.58%
Epoch [7/10], Training Loss: 0.712, Validation Accuracy: 61.98%
Epoch [8/10], Training Loss: 0.678, Validation Accuracy: 61.53%
Epoch [9/10], Training Loss: 0.660, Validation Accuracy: 61.63%
Epoch [10/10], Training Loss: 0.641, Validation Accuracy: 61.66%
Epoch [1/10], Training Loss: 0.916, Validation Accuracy: 61.13%
Epoch [2/10], Training Loss: 0.839, Validation Accuracy: 61.73%
Epoch [3/10], Training Loss: 0.787, Validation Accuracy: 61.18%
Epoch [4/10], Training Loss: 0.753, Validation Accuracy: 60.50%
Epoch [5/10], Training Loss: 0.731, Validation Accuracy: 61.84%
Epoch [6/10], Training Loss: 0.695, Validation Accuracy: 60.94%
Epoch [7/10], Training Loss: 0.674, Validation Accuracy: 60.96%
Epoch [8/10], Training Loss: 0.663, Validation Accuracy: 60.93%
Epoch [9/10], Training Loss: 0.646, Validation Accuracy: 61.35%
Epoch [10/10], Training Loss: 0.618, Validation Accuracy: 61.25%
Epoch [1/10], Training Loss: 0.915, Validation Accuracy: 60.31%
Epoch [2/10], Training Loss: 0.820, Validation Accuracy: 61.39%
Epoch [3/10], Training Loss: 0.773, Validation Accuracy: 61.22%
Epoch [4/10], Training Loss: 0.744, Validation Accuracy: 60.27%
Epoch [5/10], Training Loss: 0.711, Validation Accuracy: 60.93%
Epoch [6/10], Training Loss: 0.691, Validation Accuracy: 61.42%
Epoch [7/10], Training Loss: 0.664, Validation Accuracy: 60.34%
Epoch [8/10], Training Loss: 0.663, Validation Accuracy: 60.31%
Epoch [9/10], Training Loss: 0.634, Validation Accuracy: 60.61%
Epoch [10/10], Training Loss: 0.616, Validation Accuracy: 61.15%
Epoch [1/10], Training Loss: 0.887, Validation Accuracy: 61.10%
Epoch [2/10], Training Loss: 0.812, Validation Accuracy: 61.33%
Epoch [3/10], Training Loss: 0.761, Validation Accuracy: 61.22%
Epoch [4/10], Training Loss: 0.738, Validation Accuracy: 60.53%
Epoch [5/10], Training Loss: 0.707, Validation Accuracy: 60.70%
Epoch [6/10], Training Loss: 0.680, Validation Accuracy: 60.69%
Epoch [7/10], Training Loss: 0.683, Validation Accuracy: 60.49%
Epoch [8/10], Training Loss: 0.641, Validation Accuracy: 60.42%
Epoch [9/10], Training Loss: 0.614, Validation Accuracy: 60.22%
Epoch [10/10], Training Loss: 0.601, Validation Accuracy: 60.16%
Epoch [1/10], Training Loss: 0.900, Validation Accuracy: 60.94%
Epoch [2/10], Training Loss: 0.809, Validation Accuracy: 61.56%
Epoch [3/10], Training Loss: 0.753, Validation Accuracy: 61.37%
Epoch [4/10], Training Loss: 0.721, Validation Accuracy: 61.37%
Epoch [5/10], Training Loss: 0.689, Validation Accuracy: 61.26%
Epoch [6/10], Training Loss: 0.659, Validation Accuracy: 61.01%
Epoch [7/10], Training Loss: 0.639, Validation Accuracy: 60.89%
Epoch [8/10], Training Loss: 0.619, Validation Accuracy: 61.23%
Epoch [9/10], Training Loss: 0.609, Validation Accuracy: 60.86%
Epoch [10/10], Training Loss: 0.579, Validation Accuracy: 60.50%
"""

# Regular expression to find validation accuracies
accuracies = re.findall(r'Validation Accuracy: (\d+\.\d+)%', log)

# Convert accuracies from string to float
accuracies = [float(acc) for acc in accuracies]

# Print accuracies
print("Accuracies:", accuracies)

# Print size of the array
print("Size of array:", len(accuracies))


Accuracies: [10.04, 10.78, 11.4, 11.7, 11.99, 11.82, 11.51, 11.33, 11.38, 11.46, 11.43, 11.86, 12.51, 13.75, 14.94, 16.29, 17.4, 18.76, 20.65, 23.27, 24.55, 25.06, 25.58, 25.8, 26.4, 27.51, 28.46, 28.4, 28.93, 30.03, 31.17, 31.15, 31.97, 32.09, 32.1, 33.37, 33.2, 34.15, 34.71, 35.26, 35.33, 36.05, 35.13, 36.08, 36.72, 37.83, 37.47, 37.17, 38.33, 39.32, 39.08, 39.65, 39.7, 40.42, 40.39, 40.5, 40.47, 41.93, 41.17, 41.28, 41.96, 42.78, 42.13, 42.36, 42.97, 43.05, 43.32, 43.55, 44.07, 44.25, 43.37, 44.78, 44.92, 44.52, 45.19, 45.2, 45.03, 46.05, 45.85, 46.56, 46.45, 46.63, 47.03, 47.31, 46.22, 47.19, 47.72, 48.12, 47.44, 48.09, 48.06, 48.26, 48.7, 48.05, 48.69, 49.05, 47.7, 48.53, 49.62, 49.59, 49.24, 49.81, 50.83, 49.15, 51.02, 49.88, 50.12, 50.6, 49.96, 51.31, 50.29, 51.65, 50.89, 50.7, 52.4, 51.84, 51.16, 51.69, 52.57, 52.05, 52.26, 51.97, 52.06, 52.88, 52.62, 52.25, 53.48, 52.86, 51.74, 52.79, 53.37, 53.38, 53.78, 53.45, 54.44, 53.64, 54.34, 54.23, 52.83, 54.44, 51.78, 54.93, 54.3, 53.

In [ ]:
%%time

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()


Epoch [1/10], Training Loss: 2.303, Validation Accuracy: 9.95%
Epoch [2/10], Training Loss: 2.302, Validation Accuracy: 9.95%
Epoch [3/10], Training Loss: 2.301, Validation Accuracy: 9.95%
Epoch [4/10], Training Loss: 2.299, Validation Accuracy: 9.95%
Epoch [5/10], Training Loss: 2.298, Validation Accuracy: 9.95%
Epoch [6/10], Training Loss: 2.297, Validation Accuracy: 9.95%
Epoch [7/10], Training Loss: 2.295, Validation Accuracy: 9.95%
Epoch [8/10], Training Loss: 2.293, Validation Accuracy: 9.95%
Epoch [9/10], Training Loss: 2.290, Validation Accuracy: 9.95%
Epoch [10/10], Training Loss: 2.287, Validation Accuracy: 9.95%
Epoch [1/10], Training Loss: 2.284, Validation Accuracy: 9.95%
Epoch [2/10], Training Loss: 2.278, Validation Accuracy: 11.23%
Epoch [3/10], Training Loss: 2.268, Validation Accuracy: 15.33%
Epoch [4/10], Training Loss: 2.254, Validation Accuracy: 17.81%
Epoch [5/10], Training Loss: 2.232, Validation Accuracy: 18.93%
Epoch [6/10], Training Loss: 2.199, Validation Acc

In [ ]:
%%time

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=1)

    # Evaluate final model
    test_accuracy = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

if __name__ == "__main__":
    global_server()

Epoch [1/10], Training Loss: 2.303, Validation Accuracy: 10.19%
Epoch [2/10], Training Loss: 2.301, Validation Accuracy: 10.19%
Epoch [3/10], Training Loss: 2.299, Validation Accuracy: 10.17%
Epoch [4/10], Training Loss: 2.297, Validation Accuracy: 10.21%
Epoch [5/10], Training Loss: 2.295, Validation Accuracy: 11.53%
Epoch [6/10], Training Loss: 2.293, Validation Accuracy: 12.61%
Epoch [7/10], Training Loss: 2.289, Validation Accuracy: 13.04%
Epoch [8/10], Training Loss: 2.285, Validation Accuracy: 13.43%
Epoch [9/10], Training Loss: 2.279, Validation Accuracy: 14.08%
Epoch [10/10], Training Loss: 2.270, Validation Accuracy: 15.52%
Epoch [1/10], Training Loss: 2.261, Validation Accuracy: 19.22%
Epoch [2/10], Training Loss: 2.241, Validation Accuracy: 21.62%
Epoch [3/10], Training Loss: 2.208, Validation Accuracy: 22.73%
Epoch [4/10], Training Loss: 2.160, Validation Accuracy: 23.62%
Epoch [5/10], Training Loss: 2.107, Validation Accuracy: 25.88%
Epoch [6/10], Training Loss: 2.065, Val

In [ ]:
import numpy as np
from scipy.stats import truncnorm
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import CIFAR10
from typing import Dict, Tuple
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import random

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(epochs):
        vae.train()
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()

        scheduler.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Separate dataset by class
class_indices = {i: [] for i in range(10)}  # CIFAR-10 has 10 classes
for idx, (_, label) in enumerate(full_dataset):
    class_indices[label].append(idx)

# Define target count per class, summing to 60,000 with random distribution
class_counts = np.random.multinomial(60000, [0.1] * 10)  # Adjust probabilities if you want specific class biases
print("Random Images per Class:", class_counts)

# Sample indices based on the specified class counts
indices = []
for class_id, count in enumerate(class_counts):
    # Ensure count does not exceed available images
    count = min(count, len(class_indices[class_id]))
    selected_indices = random.sample(class_indices[class_id], count)
    indices.extend(selected_indices)

# Create a custom CIFAR-10 dataset with the sampled indices
custom_dataset = Subset(full_dataset, indices)
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)



# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    net.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    print(f"Confusion Matrix:\n{cm}")

    # Calculate TP, FP, TN, FN for each class
    tp = np.diag(cm)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (fp + fn + tp)

    # Calculate precision, recall, and F1 score for each class
    precision = precision_score(all_labels, all_predictions, average=None)
    recall = recall_score(all_labels, all_predictions, average=None)
    f1 = f1_score(all_labels, all_predictions, average=None)

    return accuracy, tp, fp, tn, fn, precision, recall, f1

# Initialize clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {

        "normal": {
            "mean": vae.encoder[-1].bias.data.cpu().numpy(),
            "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
        }
    }

    return distribution_info

def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

def generate_augmented_data(vae: VAE, distribution_info_normal: Dict) -> torch.Tensor:
    # Generate augmented data using both uniform and truncated uniform distributions



    mean_normal = distribution_info_normal["mean"]
    std_normal = distribution_info_normal["std"]

    augmented_data_normal = torch.randn(64, vae.z_dim) * std_normal + mean_normal





    # Calculate the average of augmented data from both distributions
    augmented_data_average =  augmented_data_normal

    return augmented_data_average

def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info["normal"])

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

# Define logic to receive distribution information from global server
def receive_distribution_info() -> Dict:
    # Receive distribution information logic
    distribution_info = {

        "normal": {
            "mean": np.zeros(20),  # Adjust the size based on your latent space dimension
            "std": np.ones(20)
        }
    }
    return distribution_info

def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy, tp, fp, tn, fn, precision, recall, f1 = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print("True Positives (TP):", tp)
    print("False Positives (FP):", fp)
    print("True Negatives (TN):", tn)
    print("False Negatives (FN):", fn)
    print(":", fn+tn+tp+fp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

if __name__ == "__main__":
    global_server()

Files already downloaded and verified
Files already downloaded and verified
Random Images per Class: [5958 6008 5989 5987 5944 5912 6049 6086 5993 6074]
Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 9.51%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 9.51%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 9.51%
Epoch [4/10], Training Loss: 2.303, Validation Accuracy: 9.59%
Epoch [5/10], Training Loss: 2.302, Validation Accuracy: 10.49%
Epoch [6/10], Training Loss: 2.301, Validation Accuracy: 12.03%
Epoch [7/10], Training Loss: 2.300, Validation Accuracy: 12.99%
Epoch [8/10], Training Loss: 2.299, Validation Accuracy: 13.97%
Epoch [9/10], Training Loss: 2.297, Validation Accuracy: 14.28%
Epoch [10/10], Training Loss: 2.296, Validation Accuracy: 14.70%
Epoch [1/10], Training Loss: 2.295, Validation Accuracy: 15.30%
Epoch [2/10], Training Loss: 2.293, Validation Accuracy: 16.12%
Epoch [3/10], Training Loss: 2.289, Validation Accuracy: 16.74%
Epoch [4/10], Trai

In [ ]:
import re

# Your provided text
log = """
Epoch [1/10], Training Loss: 2.306, Validation Accuracy: 9.51%
Epoch [2/10], Training Loss: 2.305, Validation Accuracy: 9.51%
Epoch [3/10], Training Loss: 2.304, Validation Accuracy: 9.51%
Epoch [4/10], Training Loss: 2.303, Validation Accuracy: 9.59%
Epoch [5/10], Training Loss: 2.302, Validation Accuracy: 10.49%
Epoch [6/10], Training Loss: 2.301, Validation Accuracy: 12.03%
Epoch [7/10], Training Loss: 2.300, Validation Accuracy: 12.99%
Epoch [8/10], Training Loss: 2.299, Validation Accuracy: 13.97%
Epoch [9/10], Training Loss: 2.297, Validation Accuracy: 14.28%
Epoch [10/10], Training Loss: 2.296, Validation Accuracy: 14.70%
Epoch [1/10], Training Loss: 2.295, Validation Accuracy: 15.30%
Epoch [2/10], Training Loss: 2.293, Validation Accuracy: 16.12%
Epoch [3/10], Training Loss: 2.289, Validation Accuracy: 16.74%
Epoch [4/10], Training Loss: 2.285, Validation Accuracy: 17.27%
Epoch [5/10], Training Loss: 2.279, Validation Accuracy: 17.83%
Epoch [6/10], Training Loss: 2.269, Validation Accuracy: 18.36%
Epoch [7/10], Training Loss: 2.254, Validation Accuracy: 18.53%
Epoch [8/10], Training Loss: 2.232, Validation Accuracy: 18.87%
Epoch [9/10], Training Loss: 2.202, Validation Accuracy: 20.00%
Epoch [10/10], Training Loss: 2.166, Validation Accuracy: 21.70%
Epoch [1/10], Training Loss: 2.146, Validation Accuracy: 24.32%
Epoch [2/10], Training Loss: 2.113, Validation Accuracy: 24.86%
Epoch [3/10], Training Loss: 2.086, Validation Accuracy: 25.75%
Epoch [4/10], Training Loss: 2.063, Validation Accuracy: 26.95%
Epoch [5/10], Training Loss: 2.042, Validation Accuracy: 27.61%
Epoch [6/10], Training Loss: 2.020, Validation Accuracy: 28.10%
Epoch [7/10], Training Loss: 1.999, Validation Accuracy: 29.06%
Epoch [8/10], Training Loss: 1.979, Validation Accuracy: 29.38%
Epoch [9/10], Training Loss: 1.962, Validation Accuracy: 30.35%
Epoch [10/10], Training Loss: 1.944, Validation Accuracy: 30.09%
Epoch [1/10], Training Loss: 1.925, Validation Accuracy: 31.08%
Epoch [2/10], Training Loss: 1.909, Validation Accuracy: 31.84%
Epoch [3/10], Training Loss: 1.894, Validation Accuracy: 32.26%
Epoch [4/10], Training Loss: 1.880, Validation Accuracy: 31.86%
Epoch [5/10], Training Loss: 1.862, Validation Accuracy: 32.98%
Epoch [6/10], Training Loss: 1.851, Validation Accuracy: 33.22%
Epoch [7/10], Training Loss: 1.831, Validation Accuracy: 34.09%
Epoch [8/10], Training Loss: 1.814, Validation Accuracy: 34.77%
Epoch [9/10], Training Loss: 1.797, Validation Accuracy: 35.25%
Epoch [10/10], Training Loss: 1.780, Validation Accuracy: 35.55%
Epoch [1/10], Training Loss: 1.784, Validation Accuracy: 36.51%
Epoch [2/10], Training Loss: 1.762, Validation Accuracy: 37.46%
Epoch [3/10], Training Loss: 1.745, Validation Accuracy: 37.44%
Epoch [4/10], Training Loss: 1.732, Validation Accuracy: 37.59%
Epoch [5/10], Training Loss: 1.717, Validation Accuracy: 38.39%
Epoch [6/10], Training Loss: 1.702, Validation Accuracy: 38.59%
Epoch [7/10], Training Loss: 1.690, Validation Accuracy: 38.62%
Epoch [8/10], Training Loss: 1.676, Validation Accuracy: 39.00%
Epoch [9/10], Training Loss: 1.664, Validation Accuracy: 40.08%
Epoch [10/10], Training Loss: 1.655, Validation Accuracy: 39.90%
Epoch [1/10], Training Loss: 1.659, Validation Accuracy: 40.48%
Epoch [2/10], Training Loss: 1.643, Validation Accuracy: 40.51%
Epoch [3/10], Training Loss: 1.633, Validation Accuracy: 40.48%
Epoch [4/10], Training Loss: 1.626, Validation Accuracy: 40.91%
Epoch [5/10], Training Loss: 1.611, Validation Accuracy: 41.18%
Epoch [6/10], Training Loss: 1.604, Validation Accuracy: 41.38%
Epoch [7/10], Training Loss: 1.597, Validation Accuracy: 41.45%
Epoch [8/10], Training Loss: 1.581, Validation Accuracy: 42.17%
Epoch [9/10], Training Loss: 1.571, Validation Accuracy: 42.23%
Epoch [10/10], Training Loss: 1.558, Validation Accuracy: 42.01%
Epoch [1/10], Training Loss: 1.582, Validation Accuracy: 43.41%
Epoch [2/10], Training Loss: 1.572, Validation Accuracy: 42.65%
Epoch [3/10], Training Loss: 1.557, Validation Accuracy: 42.76%
Epoch [4/10], Training Loss: 1.561, Validation Accuracy: 43.47%
Epoch [5/10], Training Loss: 1.536, Validation Accuracy: 43.94%
Epoch [6/10], Training Loss: 1.527, Validation Accuracy: 43.97%
Epoch [7/10], Training Loss: 1.522, Validation Accuracy: 44.44%
Epoch [8/10], Training Loss: 1.517, Validation Accuracy: 44.25%
Epoch [9/10], Training Loss: 1.504, Validation Accuracy: 43.81%
Epoch [10/10], Training Loss: 1.495, Validation Accuracy: 44.55%
Epoch [1/10], Training Loss: 1.534, Validation Accuracy: 44.97%
Epoch [2/10], Training Loss: 1.522, Validation Accuracy: 45.03%
Epoch [3/10], Training Loss: 1.507, Validation Accuracy: 45.36%
Epoch [4/10], Training Loss: 1.499, Validation Accuracy: 45.93%
Epoch [5/10], Training Loss: 1.488, Validation Accuracy: 45.76%
Epoch [6/10], Training Loss: 1.478, Validation Accuracy: 45.61%
Epoch [7/10], Training Loss: 1.472, Validation Accuracy: 46.72%
Epoch [8/10], Training Loss: 1.465, Validation Accuracy: 46.76%
Epoch [9/10], Training Loss: 1.452, Validation Accuracy: 46.02%
Epoch [10/10], Training Loss: 1.443, Validation Accuracy: 46.23%
Epoch [1/10], Training Loss: 1.467, Validation Accuracy: 46.79%
Epoch [2/10], Training Loss: 1.452, Validation Accuracy: 47.67%
Epoch [3/10], Training Loss: 1.446, Validation Accuracy: 47.21%
Epoch [4/10], Training Loss: 1.431, Validation Accuracy: 47.32%
Epoch [5/10], Training Loss: 1.420, Validation Accuracy: 47.66%
Epoch [6/10], Training Loss: 1.411, Validation Accuracy: 48.43%
Epoch [7/10], Training Loss: 1.401, Validation Accuracy: 48.33%
Epoch [8/10], Training Loss: 1.392, Validation Accuracy: 48.17%
Epoch [9/10], Training Loss: 1.389, Validation Accuracy: 48.20%
Epoch [10/10], Training Loss: 1.378, Validation Accuracy: 47.38%
Epoch [1/10], Training Loss: 1.406, Validation Accuracy: 48.92%
Epoch [2/10], Training Loss: 1.386, Validation Accuracy: 49.72%
Epoch [3/10], Training Loss: 1.386, Validation Accuracy: 49.88%
Epoch [4/10], Training Loss: 1.372, Validation Accuracy: 50.11%
Epoch [5/10], Training Loss: 1.357, Validation Accuracy: 49.05%
Epoch [6/10], Training Loss: 1.348, Validation Accuracy: 49.04%
Epoch [7/10], Training Loss: 1.343, Validation Accuracy: 49.55%
Epoch [8/10], Training Loss: 1.326, Validation Accuracy: 50.44%
Epoch [9/10], Training Loss: 1.320, Validation Accuracy: 50.79%
Epoch [10/10], Training Loss: 1.312, Validation Accuracy: 50.36%
Epoch [1/10], Training Loss: 1.365, Validation Accuracy: 50.95%
Epoch [2/10], Training Loss: 1.350, Validation Accuracy: 51.36%
Epoch [3/10], Training Loss: 1.331, Validation Accuracy: 50.44%
Epoch [4/10], Training Loss: 1.322, Validation Accuracy: 51.21%
Epoch [5/10], Training Loss: 1.311, Validation Accuracy: 50.53%
Epoch [6/10], Training Loss: 1.305, Validation Accuracy: 51.40%
Epoch [7/10], Training Loss: 1.295, Validation Accuracy: 51.92%
Epoch [8/10], Training Loss: 1.287, Validation Accuracy: 50.25%
Epoch [9/10], Training Loss: 1.286, Validation Accuracy: 51.02%
Epoch [10/10], Training Loss: 1.274, Validation Accuracy: 51.68%
Epoch [1/10], Training Loss: 1.343, Validation Accuracy: 52.63%
Epoch [2/10], Training Loss: 1.322, Validation Accuracy: 52.26%
Epoch [3/10], Training Loss: 1.313, Validation Accuracy: 52.45%
Epoch [4/10], Training Loss: 1.302, Validation Accuracy: 52.38%
Epoch [5/10], Training Loss: 1.298, Validation Accuracy: 52.19%
Epoch [6/10], Training Loss: 1.283, Validation Accuracy: 53.00%
Epoch [7/10], Training Loss: 1.271, Validation Accuracy: 51.67%
Epoch [8/10], Training Loss: 1.261, Validation Accuracy: 52.86%
Epoch [9/10], Training Loss: 1.253, Validation Accuracy: 53.14%
Epoch [10/10], Training Loss: 1.237, Validation Accuracy: 52.92%
Epoch [1/10], Training Loss: 1.311, Validation Accuracy: 53.14%
Epoch [2/10], Training Loss: 1.306, Validation Accuracy: 53.21%
Epoch [3/10], Training Loss: 1.278, Validation Accuracy: 52.94%
Epoch [4/10], Training Loss: 1.274, Validation Accuracy: 53.15%
Epoch [5/10], Training Loss: 1.268, Validation Accuracy: 53.72%
Epoch [6/10], Training Loss: 1.250, Validation Accuracy: 53.89%
Epoch [7/10], Training Loss: 1.245, Validation Accuracy: 53.52%
Epoch [8/10], Training Loss: 1.240, Validation Accuracy: 53.97%
Epoch [9/10], Training Loss: 1.224, Validation Accuracy: 54.15%
Epoch [10/10], Training Loss: 1.220, Validation Accuracy: 54.06%
Epoch [1/10], Training Loss: 1.282, Validation Accuracy: 54.34%
Epoch [2/10], Training Loss: 1.259, Validation Accuracy: 54.74%
Epoch [3/10], Training Loss: 1.245, Validation Accuracy: 53.76%
Epoch [4/10], Training Loss: 1.237, Validation Accuracy: 54.03%
Epoch [5/10], Training Loss: 1.226, Validation Accuracy: 53.63%
Epoch [6/10], Training Loss: 1.216, Validation Accuracy: 54.41%
Epoch [7/10], Training Loss: 1.210, Validation Accuracy: 54.29%
Epoch [8/10], Training Loss: 1.205, Validation Accuracy: 54.23%
Epoch [9/10], Training Loss: 1.196, Validation Accuracy: 54.45%
Epoch [10/10], Training Loss: 1.185, Validation Accuracy: 53.70%
Epoch [1/10], Training Loss: 1.233, Validation Accuracy: 54.79%
Epoch [2/10], Training Loss: 1.213, Validation Accuracy: 55.20%
Epoch [3/10], Training Loss: 1.197, Validation Accuracy: 53.42%
Epoch [4/10], Training Loss: 1.197, Validation Accuracy: 55.64%
Epoch [5/10], Training Loss: 1.179, Validation Accuracy: 55.25%
Epoch [6/10], Training Loss: 1.166, Validation Accuracy: 55.39%
Epoch [7/10], Training Loss: 1.164, Validation Accuracy: 54.19%
Epoch [8/10], Training Loss: 1.159, Validation Accuracy: 55.59%
Epoch [9/10], Training Loss: 1.132, Validation Accuracy: 55.81%
Epoch [10/10], Training Loss: 1.127, Validation Accuracy: 55.44%
Epoch [1/10], Training Loss: 1.212, Validation Accuracy: 55.85%
Epoch [2/10], Training Loss: 1.190, Validation Accuracy: 55.67%
Epoch [3/10], Training Loss: 1.174, Validation Accuracy: 56.20%
Epoch [4/10], Training Loss: 1.159, Validation Accuracy: 55.16%
Epoch [5/10], Training Loss: 1.145, Validation Accuracy: 55.85%
Epoch [6/10], Training Loss: 1.143, Validation Accuracy: 55.87%
Epoch [7/10], Training Loss: 1.133, Validation Accuracy: 55.71%
Epoch [8/10], Training Loss: 1.127, Validation Accuracy: 56.39%
Epoch [9/10], Training Loss: 1.114, Validation Accuracy: 56.30%
Epoch [10/10], Training Loss: 1.104, Validation Accuracy: 56.14%
Epoch [1/10], Training Loss: 1.209, Validation Accuracy: 56.19%
Epoch [2/10], Training Loss: 1.199, Validation Accuracy: 56.12%
Epoch [3/10], Training Loss: 1.172, Validation Accuracy: 56.08%
Epoch [4/10], Training Loss: 1.158, Validation Accuracy: 57.09%
Epoch [5/10], Training Loss: 1.145, Validation Accuracy: 56.68%
Epoch [6/10], Training Loss: 1.134, Validation Accuracy: 56.55%
Epoch [7/10], Training Loss: 1.133, Validation Accuracy: 56.93%
Epoch [8/10], Training Loss: 1.126, Validation Accuracy: 57.59%
Epoch [9/10], Training Loss: 1.104, Validation Accuracy: 57.07%
Epoch [10/10], Training Loss: 1.112, Validation Accuracy: 57.31%
Epoch [1/10], Training Loss: 1.186, Validation Accuracy: 57.57%
Epoch [2/10], Training Loss: 1.161, Validation Accuracy: 56.54%
Epoch [3/10], Training Loss: 1.153, Validation Accuracy: 57.64%
Epoch [4/10], Training Loss: 1.137, Validation Accuracy: 57.42%
Epoch [5/10], Training Loss: 1.120, Validation Accuracy: 57.53%
Epoch [6/10], Training Loss: 1.115, Validation Accuracy: 57.47%
Epoch [7/10], Training Loss: 1.106, Validation Accuracy: 57.46%
Epoch [8/10], Training Loss: 1.091, Validation Accuracy: 57.69%
Epoch [9/10], Training Loss: 1.089, Validation Accuracy: 57.81%
Epoch [10/10], Training Loss: 1.082, Validation Accuracy: 57.64%
Epoch [1/10], Training Loss: 1.172, Validation Accuracy: 58.00%
Epoch [2/10], Training Loss: 1.141, Validation Accuracy: 57.27%
Epoch [3/10], Training Loss: 1.138, Validation Accuracy: 57.83%
Epoch [4/10], Training Loss: 1.114, Validation Accuracy: 58.23%
Epoch [5/10], Training Loss: 1.106, Validation Accuracy: 57.27%
Epoch [6/10], Training Loss: 1.088, Validation Accuracy: 58.18%
Epoch [7/10], Training Loss: 1.078, Validation Accuracy: 57.80%
Epoch [8/10], Training Loss: 1.067, Validation Accuracy: 57.71%
Epoch [9/10], Training Loss: 1.060, Validation Accuracy: 57.90%
Epoch [10/10], Training Loss: 1.053, Validation Accuracy: 58.06%
Epoch [1/10], Training Loss: 1.125, Validation Accuracy: 57.94%
Epoch [2/10], Training Loss: 1.099, Validation Accuracy: 58.49%
Epoch [3/10], Training Loss: 1.083, Validation Accuracy: 58.68%
Epoch [4/10], Training Loss: 1.064, Validation Accuracy: 58.72%
Epoch [5/10], Training Loss: 1.052, Validation Accuracy: 59.11%
Epoch [6/10], Training Loss: 1.042, Validation Accuracy: 57.30%
Epoch [7/10], Training Loss: 1.029, Validation Accuracy: 58.04%
Epoch [8/10], Training Loss: 1.015, Validation Accuracy: 58.87%
Epoch [9/10], Training Loss: 1.008, Validation Accuracy: 58.25%
Epoch [10/10], Training Loss: 1.002, Validation Accuracy: 57.94%
Epoch [1/10], Training Loss: 1.113, Validation Accuracy: 58.78%
Epoch [2/10], Training Loss: 1.080, Validation Accuracy: 58.71%
Epoch [3/10], Training Loss: 1.065, Validation Accuracy: 59.09%
Epoch [4/10], Training Loss: 1.042, Validation Accuracy: 59.39%
Epoch [5/10], Training Loss: 1.024, Validation Accuracy: 58.93%
Epoch [6/10], Training Loss: 1.016, Validation Accuracy: 58.19%
Epoch [7/10], Training Loss: 1.010, Validation Accuracy: 58.83%
Epoch [8/10], Training Loss: 0.997, Validation Accuracy: 59.66%
Epoch [9/10], Training Loss: 0.986, Validation Accuracy: 58.70%
Epoch [10/10], Training Loss: 0.972, Validation Accuracy: 58.19%
Epoch [1/10], Training Loss: 1.126, Validation Accuracy: 59.31%
Epoch [2/10], Training Loss: 1.088, Validation Accuracy: 59.96%
Epoch [3/10], Training Loss: 1.069, Validation Accuracy: 59.70%
Epoch [4/10], Training Loss: 1.056, Validation Accuracy: 59.86%
Epoch [5/10], Training Loss: 1.044, Validation Accuracy: 59.55%
Epoch [6/10], Training Loss: 1.038, Validation Accuracy: 58.08%
Epoch [7/10], Training Loss: 1.027, Validation Accuracy: 59.93%
Epoch [8/10], Training Loss: 1.002, Validation Accuracy: 59.44%
Epoch [9/10], Training Loss: 0.990, Validation Accuracy: 58.90%
Epoch [10/10], Training Loss: 0.984, Validation Accuracy: 58.13%
Epoch [1/10], Training Loss: 1.102, Validation Accuracy: 58.76%
Epoch [2/10], Training Loss: 1.076, Validation Accuracy: 59.55%
Epoch [3/10], Training Loss: 1.043, Validation Accuracy: 59.86%
Epoch [4/10], Training Loss: 1.031, Validation Accuracy: 59.30%
Epoch [5/10], Training Loss: 1.022, Validation Accuracy: 59.75%
Epoch [6/10], Training Loss: 1.007, Validation Accuracy: 58.49%
Epoch [7/10], Training Loss: 0.991, Validation Accuracy: 58.59%
Epoch [8/10], Training Loss: 0.988, Validation Accuracy: 59.44%
Epoch [9/10], Training Loss: 0.965, Validation Accuracy: 59.98%
Epoch [10/10], Training Loss: 0.947, Validation Accuracy: 59.53%
Epoch [1/10], Training Loss: 1.089, Validation Accuracy: 59.27%
Epoch [2/10], Training Loss: 1.055, Validation Accuracy: 59.94%
Epoch [3/10], Training Loss: 1.031, Validation Accuracy: 59.95%
Epoch [4/10], Training Loss: 1.006, Validation Accuracy: 59.91%
Epoch [5/10], Training Loss: 0.994, Validation Accuracy: 59.20%
Epoch [6/10], Training Loss: 0.985, Validation Accuracy: 59.75%
Epoch [7/10], Training Loss: 0.974, Validation Accuracy: 60.03%
Epoch [8/10], Training Loss: 0.955, Validation Accuracy: 59.35%
Epoch [9/10], Training Loss: 0.942, Validation Accuracy: 59.77%
Epoch [10/10], Training Loss: 0.938, Validation Accuracy: 59.13%
Epoch [1/10], Training Loss: 1.044, Validation Accuracy: 59.17%
Epoch [2/10], Training Loss: 1.013, Validation Accuracy: 60.25%
Epoch [3/10], Training Loss: 0.992, Validation Accuracy: 60.72%
Epoch [4/10], Training Loss: 0.975, Validation Accuracy: 60.27%
Epoch [5/10], Training Loss: 0.954, Validation Accuracy: 60.47%
Epoch [6/10], Training Loss: 0.939, Validation Accuracy: 60.17%
Epoch [7/10], Training Loss: 0.921, Validation Accuracy: 60.02%
Epoch [8/10], Training Loss: 0.906, Validation Accuracy: 60.16%
Epoch [9/10], Training Loss: 0.897, Validation Accuracy: 60.23%
Epoch [10/10], Training Loss: 0.877, Validation Accuracy: 60.38%
Epoch [1/10], Training Loss: 1.043, Validation Accuracy: 60.15%
Epoch [2/10], Training Loss: 0.995, Validation Accuracy: 60.43%
Epoch [3/10], Training Loss: 0.973, Validation Accuracy: 60.09%
Epoch [4/10], Training Loss: 0.951, Validation Accuracy: 60.76%
Epoch [5/10], Training Loss: 0.928, Validation Accuracy: 60.58%
Epoch [6/10], Training Loss: 0.912, Validation Accuracy: 61.15%
Epoch [7/10], Training Loss: 0.901, Validation Accuracy: 60.06%
Epoch [8/10], Training Loss: 0.882, Validation Accuracy: 59.91%
Epoch [9/10], Training Loss: 0.880, Validation Accuracy: 60.56%
Epoch [10/10], Training Loss: 0.871, Validation Accuracy: 60.47%
Epoch [1/10], Training Loss: 1.063, Validation Accuracy: 60.27%
Epoch [2/10], Training Loss: 1.009, Validation Accuracy: 60.52%
Epoch [3/10], Training Loss: 0.994, Validation Accuracy: 60.74%
Epoch [4/10], Training Loss: 0.968, Validation Accuracy: 60.82%
Epoch [5/10], Training Loss: 0.955, Validation Accuracy: 60.65%
Epoch [6/10], Training Loss: 0.932, Validation Accuracy: 60.35%
Epoch [7/10], Training Loss: 0.917, Validation Accuracy: 60.81%
Epoch [8/10], Training Loss: 0.906, Validation Accuracy: 60.67%
Epoch [9/10], Training Loss: 0.891, Validation Accuracy: 60.01%
Epoch [10/10], Training Loss: 0.890, Validation Accuracy: 59.58%
Epoch [1/10], Training Loss: 1.031, Validation Accuracy: 60.43%
Epoch [2/10], Training Loss: 0.979, Validation Accuracy: 60.98%
Epoch [3/10], Training Loss: 0.964, Validation Accuracy: 59.61%
Epoch [4/10], Training Loss: 0.942, Validation Accuracy: 59.95%
Epoch [5/10], Training Loss: 0.925, Validation Accuracy: 60.39%
Epoch [6/10], Training Loss: 0.913, Validation Accuracy: 60.94%
Epoch [7/10], Training Loss: 0.883, Validation Accuracy: 60.88%
Epoch [8/10], Training Loss: 0.871, Validation Accuracy: 60.17%
Epoch [9/10], Training Loss: 0.874, Validation Accuracy: 60.59%
Epoch [10/10], Training Loss: 0.844, Validation Accuracy: 60.53%
Epoch [1/10], Training Loss: 1.018, Validation Accuracy: 60.44%
Epoch [2/10], Training Loss: 0.971, Validation Accuracy: 61.09%
Epoch [3/10], Training Loss: 0.943, Validation Accuracy: 61.32%
Epoch [4/10], Training Loss: 0.923, Validation Accuracy: 61.12%
Epoch [5/10], Training Loss: 0.902, Validation Accuracy: 61.09%
Epoch [6/10], Training Loss: 0.885, Validation Accuracy: 60.90%
Epoch [7/10], Training Loss: 0.864, Validation Accuracy: 60.75%
Epoch [8/10], Training Loss: 0.851, Validation Accuracy: 60.42%
Epoch [9/10], Training Loss: 0.840, Validation Accuracy: 60.77%
Epoch [10/10], Training Loss: 0.827, Validation Accuracy: 60.23%
Epoch [1/10], Training Loss: 0.982, Validation Accuracy: 61.12%
Epoch [2/10], Training Loss: 0.933, Validation Accuracy: 61.41%
Epoch [3/10], Training Loss: 0.906, Validation Accuracy: 60.90%
Epoch [4/10], Training Loss: 0.890, Validation Accuracy: 61.49%
Epoch [5/10], Training Loss: 0.859, Validation Accuracy: 61.04%
Epoch [6/10], Training Loss: 0.842, Validation Accuracy: 60.82%
Epoch [7/10], Training Loss: 0.820, Validation Accuracy: 61.16%
Epoch [8/10], Training Loss: 0.813, Validation Accuracy: 61.23%
Epoch [9/10], Training Loss: 0.806, Validation Accuracy: 60.69%
Epoch [10/10], Training Loss: 0.782, Validation Accuracy: 61.05%
Epoch [1/10], Training Loss: 0.975, Validation Accuracy: 61.19%
Epoch [2/10], Training Loss: 0.921, Validation Accuracy: 61.06%
Epoch [3/10], Training Loss: 0.895, Validation Accuracy: 60.55%
Epoch [4/10], Training Loss: 0.871, Validation Accuracy: 61.68%
Epoch [5/10], Training Loss: 0.844, Validation Accuracy: 61.40%
Epoch [6/10], Training Loss: 0.829, Validation Accuracy: 61.48%
Epoch [7/10], Training Loss: 0.811, Validation Accuracy: 60.88%
Epoch [8/10], Training Loss: 0.792, Validation Accuracy: 61.33%
Epoch [9/10], Training Loss: 0.769, Validation Accuracy: 61.63%
Epoch [10/10], Training Loss: 0.764, Validation Accuracy: 60.62%
Epoch [1/10], Training Loss: 0.996, Validation Accuracy: 60.33%
Epoch [2/10], Training Loss: 0.946, Validation Accuracy: 60.96%
Epoch [3/10], Training Loss: 0.911, Validation Accuracy: 61.92%
Epoch [4/10], Training Loss: 0.891, Validation Accuracy: 61.31%
Epoch [5/10], Training Loss: 0.861, Validation Accuracy: 61.47%
Epoch [6/10], Training Loss: 0.836, Validation Accuracy: 61.12%
Epoch [7/10], Training Loss: 0.820, Validation Accuracy: 61.45%
Epoch [8/10], Training Loss: 0.810, Validation Accuracy: 61.63%
Epoch [9/10], Training Loss: 0.791, Validation Accuracy: 61.24%
Epoch [10/10], Training Loss: 0.773, Validation Accuracy: 61.58%
Epoch [1/10], Training Loss: 0.965, Validation Accuracy: 61.55%
Epoch [2/10], Training Loss: 0.909, Validation Accuracy: 61.32%
Epoch [3/10], Training Loss: 0.886, Validation Accuracy: 61.03%
Epoch [4/10], Training Loss: 0.863, Validation Accuracy: 61.52%
Epoch [5/10], Training Loss: 0.825, Validation Accuracy: 61.33%
Epoch [6/10], Training Loss: 0.814, Validation Accuracy: 61.06%
Epoch [7/10], Training Loss: 0.790, Validation Accuracy: 61.23%
Epoch [8/10], Training Loss: 0.780, Validation Accuracy: 61.36%
Epoch [9/10], Training Loss: 0.764, Validation Accuracy: 61.24%
Epoch [10/10], Training Loss: 0.747, Validation Accuracy: 61.43%
Epoch [1/10], Training Loss: 0.970, Validation Accuracy: 61.38%
Epoch [2/10], Training Loss: 0.902, Validation Accuracy: 62.31%
Epoch [3/10], Training Loss: 0.870, Validation Accuracy: 61.77%
Epoch [4/10], Training Loss: 0.839, Validation Accuracy: 62.31%
Epoch [5/10], Training Loss: 0.811, Validation Accuracy: 61.80%
Epoch [6/10], Training Loss: 0.797, Validation Accuracy: 62.17%
Epoch [7/10], Training Loss: 0.784, Validation Accuracy: 61.07%
Epoch [8/10], Training Loss: 0.764, Validation Accuracy: 61.49%
Epoch [9/10], Training Loss: 0.742, Validation Accuracy: 61.15%
Epoch [10/10], Training Loss: 0.736, Validation Accuracy: 60.82%
Epoch [1/10], Training Loss: 0.936, Validation Accuracy: 60.93%
Epoch [2/10], Training Loss: 0.878, Validation Accuracy: 62.41%
Epoch [3/10], Training Loss: 0.833, Validation Accuracy: 62.69%
Epoch [4/10], Training Loss: 0.800, Validation Accuracy: 62.02%
Epoch [5/10], Training Loss: 0.778, Validation Accuracy: 62.16%
Epoch [6/10], Training Loss: 0.752, Validation Accuracy: 61.87%
Epoch [7/10], Training Loss: 0.742, Validation Accuracy: 61.71%
Epoch [8/10], Training Loss: 0.722, Validation Accuracy: 61.93%
Epoch [9/10], Training Loss: 0.705, Validation Accuracy: 61.30%
Epoch [10/10], Training Loss: 0.691, Validation Accuracy: 60.98%
Epoch [1/10], Training Loss: 0.926, Validation Accuracy: 61.44%
Epoch [2/10], Training Loss: 0.846, Validation Accuracy: 62.36%
Epoch [3/10], Training Loss: 0.811, Validation Accuracy: 62.33%
Epoch [4/10], Training Loss: 0.779, Validation Accuracy: 62.16%
Epoch [5/10], Training Loss: 0.762, Validation Accuracy: 61.08%
Epoch [6/10], Training Loss: 0.743, Validation Accuracy: 61.83%
Epoch [7/10], Training Loss: 0.723, Validation Accuracy: 61.51%
Epoch [8/10], Training Loss: 0.696, Validation Accuracy: 62.06%
Epoch [9/10], Training Loss: 0.678, Validation Accuracy: 61.90%
Epoch [10/10], Training Loss: 0.663, Validation Accuracy: 61.97%
Epoch [1/10], Training Loss: 0.945, Validation Accuracy: 61.54%
Epoch [2/10], Training Loss: 0.881, Validation Accuracy: 61.39%
Epoch [3/10], Training Loss: 0.837, Validation Accuracy: 61.90%
Epoch [4/10], Training Loss: 0.805, Validation Accuracy: 61.33%
Epoch [5/10], Training Loss: 0.777, Validation Accuracy: 62.31%
Epoch [6/10], Training Loss: 0.757, Validation Accuracy: 61.55%
Epoch [7/10], Training Loss: 0.735, Validation Accuracy: 61.63%
Epoch [8/10], Training Loss: 0.712, Validation Accuracy: 62.24%
Epoch [9/10], Training Loss: 0.697, Validation Accuracy: 61.41%
Epoch [10/10], Training Loss: 0.686, Validation Accuracy: 61.84%
Epoch [1/10], Training Loss: 0.922, Validation Accuracy: 61.28%
Epoch [2/10], Training Loss: 0.848, Validation Accuracy: 61.48%
Epoch [3/10], Training Loss: 0.825, Validation Accuracy: 62.27%
Epoch [4/10], Training Loss: 0.784, Validation Accuracy: 62.17%
Epoch [5/10], Training Loss: 0.761, Validation Accuracy: 62.25%
Epoch [6/10], Training Loss: 0.737, Validation Accuracy: 60.61%
Epoch [7/10], Training Loss: 0.720, Validation Accuracy: 61.69%
Epoch [8/10], Training Loss: 0.692, Validation Accuracy: 61.58%
Epoch [9/10], Training Loss: 0.668, Validation Accuracy: 61.83%
Epoch [10/10], Training Loss: 0.665, Validation Accuracy: 61.25%
Epoch [1/10], Training Loss: 0.922, Validation Accuracy: 61.70%
Epoch [2/10], Training Loss: 0.837, Validation Accuracy: 61.85%
Epoch [3/10], Training Loss: 0.794, Validation Accuracy: 62.84%
Epoch [4/10], Training Loss: 0.773, Validation Accuracy: 61.87%
Epoch [5/10], Training Loss: 0.739, Validation Accuracy: 62.10%
Epoch [6/10], Training Loss: 0.710, Validation Accuracy: 61.41%
Epoch [7/10], Training Loss: 0.688, Validation Accuracy: 62.39%
Epoch [8/10], Training Loss: 0.665, Validation Accuracy: 62.18%
Epoch [9/10], Training Loss: 0.658, Validation Accuracy: 62.61%
Epoch [10/10], Training Loss: 0.638, Validation Accuracy: 62.01%
Epoch [1/10], Training Loss: 0.897, Validation Accuracy: 63.00%
Epoch [2/10], Training Loss: 0.809, Validation Accuracy: 61.70%
Epoch [3/10], Training Loss: 0.759, Validation Accuracy: 61.86%
Epoch [4/10], Training Loss: 0.734, Validation Accuracy: 60.95%
Epoch [5/10], Training Loss: 0.698, Validation Accuracy: 62.54%
Epoch [6/10], Training Loss: 0.679, Validation Accuracy: 62.85%
Epoch [7/10], Training Loss: 0.657, Validation Accuracy: 60.45%
Epoch [8/10], Training Loss: 0.641, Validation Accuracy: 62.05%
Epoch [9/10], Training Loss: 0.623, Validation Accuracy: 62.10%
Epoch [10/10], Training Loss: 0.594, Validation Accuracy: 62.47%
Epoch [1/10], Training Loss: 0.872, Validation Accuracy: 62.12%
Epoch [2/10], Training Loss: 0.789, Validation Accuracy: 62.54%
Epoch [3/10], Training Loss: 0.739, Validation Accuracy: 62.21%
Epoch [4/10], Training Loss: 0.707, Validation Accuracy: 62.45%
Epoch [5/10], Training Loss: 0.680, Validation Accuracy: 62.25%
Epoch [6/10], Training Loss: 0.652, Validation Accuracy: 61.38%
Epoch [7/10], Training Loss: 0.637, Validation Accuracy: 61.20%
Epoch [8/10], Training Loss: 0.620, Validation Accuracy: 61.50%
Epoch [9/10], Training Loss: 0.599, Validation Accuracy: 61.68%
Epoch [10/10], Training Loss: 0.570, Validation Accuracy: 61.89%
Epoch [1/10], Training Loss: 0.905, Validation Accuracy: 61.09%
Epoch [2/10], Training Loss: 0.820, Validation Accuracy: 61.28%
Epoch [3/10], Training Loss: 0.769, Validation Accuracy: 61.61%
Epoch [4/10], Training Loss: 0.728, Validation Accuracy: 61.71%
Epoch [5/10], Training Loss: 0.704, Validation Accuracy: 61.90%
Epoch [6/10], Training Loss: 0.673, Validation Accuracy: 62.01%
Epoch [7/10], Training Loss: 0.659, Validation Accuracy: 61.79%
Epoch [8/10], Training Loss: 0.632, Validation Accuracy: 61.28%
Epoch [9/10], Training Loss: 0.610, Validation Accuracy: 61.72%
Epoch [10/10], Training Loss: 0.596, Validation Accuracy: 61.65%
Epoch [1/10], Training Loss: 0.881, Validation Accuracy: 61.78%
Epoch [2/10], Training Loss: 0.790, Validation Accuracy: 62.74%
Epoch [3/10], Training Loss: 0.751, Validation Accuracy: 61.89%
Epoch [4/10], Training Loss: 0.709, Validation Accuracy: 62.36%
Epoch [5/10], Training Loss: 0.679, Validation Accuracy: 62.76%
Epoch [6/10], Training Loss: 0.663, Validation Accuracy: 62.06%
Epoch [7/10], Training Loss: 0.634, Validation Accuracy: 61.53%
Epoch [8/10], Training Loss: 0.617, Validation Accuracy: 61.60%
Epoch [9/10], Training Loss: 0.595, Validation Accuracy: 61.70%
Epoch [10/10], Training Loss: 0.574, Validation Accuracy: 62.06%
Epoch [1/10], Training Loss: 0.880, Validation Accuracy: 62.42%
Epoch [2/10], Training Loss: 0.784, Validation Accuracy: 62.73%
Epoch [3/10], Training Loss: 0.722, Validation Accuracy: 62.09%
Epoch [4/10], Training Loss: 0.688, Validation Accuracy: 62.86%
Epoch [5/10], Training Loss: 0.670, Validation Accuracy: 62.42%
Epoch [6/10], Training Loss: 0.626, Validation Accuracy: 62.40%
Epoch [7/10], Training Loss: 0.607, Validation Accuracy: 62.65%
Epoch [8/10], Training Loss: 0.604, Validation Accuracy: 61.53%
Epoch [9/10], Training Loss: 0.572, Validation Accuracy: 62.48%
Epoch [10/10], Training Loss: 0.549, Validation Accuracy: 61.88%
Epoch [1/10], Training Loss: 0.843, Validation Accuracy: 62.33%
Epoch [2/10], Training Loss: 0.743, Validation Accuracy: 62.76%
Epoch [3/10], Training Loss: 0.697, Validation Accuracy: 62.69%
Epoch [4/10], Training Loss: 0.659, Validation Accuracy: 61.87%
Epoch [5/10], Training Loss: 0.623, Validation Accuracy: 62.07%
Epoch [6/10], Training Loss: 0.596, Validation Accuracy: 62.07%
Epoch [7/10], Training Loss: 0.580, Validation Accuracy: 62.87%
Epoch [8/10], Training Loss: 0.548, Validation Accuracy: 62.49%
Epoch [9/10], Training Loss: 0.536, Validation Accuracy: 61.93%
Epoch [10/10], Training Loss: 0.512, Validation Accuracy: 62.40%
Epoch [1/10], Training Loss: 0.832, Validation Accuracy: 62.07%
Epoch [2/10], Training Loss: 0.720, Validation Accuracy: 62.74%
Epoch [3/10], Training Loss: 0.674, Validation Accuracy: 61.73%
Epoch [4/10], Training Loss: 0.636, Validation Accuracy: 62.13%
Epoch [5/10], Training Loss: 0.605, Validation Accuracy: 61.86%
Epoch [6/10], Training Loss: 0.575, Validation Accuracy: 62.42%
Epoch [7/10], Training Loss: 0.544, Validation Accuracy: 62.64%
Epoch [8/10], Training Loss: 0.525, Validation Accuracy: 61.65%
Epoch [9/10], Training Loss: 0.508, Validation Accuracy: 61.19%
Epoch [10/10], Training Loss: 0.498, Validation Accuracy: 61.96%
Epoch [1/10], Training Loss: 0.865, Validation Accuracy: 61.54%
Epoch [2/10], Training Loss: 0.758, Validation Accuracy: 62.08%
Epoch [3/10], Training Loss: 0.692, Validation Accuracy: 61.79%
Epoch [4/10], Training Loss: 0.657, Validation Accuracy: 61.18%
Epoch [5/10], Training Loss: 0.628, Validation Accuracy: 61.69%
Epoch [6/10], Training Loss: 0.593, Validation Accuracy: 61.98%
Epoch [7/10], Training Loss: 0.568, Validation Accuracy: 60.76%
Epoch [8/10], Training Loss: 0.550, Validation Accuracy: 62.00%
Epoch [9/10], Training Loss: 0.526, Validation Accuracy: 60.35%
Epoch [10/10], Training Loss: 0.508, Validation Accuracy: 60.83%
Epoch [1/10], Training Loss: 0.841, Validation Accuracy: 61.18%
Epoch [2/10], Training Loss: 0.731, Validation Accuracy: 61.77%
Epoch [3/10], Training Loss: 0.678, Validation Accuracy: 61.88%
Epoch [4/10], Training Loss: 0.643, Validation Accuracy: 62.31%
Epoch [5/10], Training Loss: 0.596, Validation Accuracy: 62.35%
Epoch [6/10], Training Loss: 0.575, Validation Accuracy: 61.82%
Epoch [7/10], Training Loss: 0.545, Validation Accuracy: 62.19%
Epoch [8/10], Training Loss: 0.533, Validation Accuracy: 61.81%
Epoch [9/10], Training Loss: 0.497, Validation Accuracy: 61.63%
Epoch [10/10], Training Loss: 0.482, Validation Accuracy: 61.56%
Epoch [1/10], Training Loss: 0.861, Validation Accuracy: 61.68%
Epoch [2/10], Training Loss: 0.723, Validation Accuracy: 62.32%
Epoch [3/10], Training Loss: 0.671, Validation Accuracy: 62.24%
Epoch [4/10], Training Loss: 0.617, Validation Accuracy: 62.34%
Epoch [5/10], Training Loss: 0.582, Validation Accuracy: 62.14%
Epoch [6/10], Training Loss: 0.558, Validation Accuracy: 62.44%
Epoch [7/10], Training Loss: 0.529, Validation Accuracy: 62.36%
Epoch [8/10], Training Loss: 0.506, Validation Accuracy: 62.31%
Epoch [9/10], Training Loss: 0.487, Validation Accuracy: 61.77%
Epoch [10/10], Training Loss: 0.481, Validation Accuracy: 61.43%
Epoch [1/10], Training Loss: 0.812, Validation Accuracy: 62.16%
Epoch [2/10], Training Loss: 0.698, Validation Accuracy: 61.64%
Epoch [3/10], Training Loss: 0.623, Validation Accuracy: 62.20%
Epoch [4/10], Training Loss: 0.589, Validation Accuracy: 61.81%
Epoch [5/10], Training Loss: 0.557, Validation Accuracy: 61.91%
Epoch [6/10], Training Loss: 0.527, Validation Accuracy: 62.58%
Epoch [7/10], Training Loss: 0.503, Validation Accuracy: 61.61%
Epoch [8/10], Training Loss: 0.485, Validation Accuracy: 62.11%
Epoch [9/10], Training Loss: 0.458, Validation Accuracy: 62.00%
Epoch [10/10], Training Loss: 0.432, Validation Accuracy: 62.45%
"""

# Regular expression to find validation accuracies
accuracies = re.findall(r'Validation Accuracy: (\d+\.\d+)%', log)

# Convert accuracies from string to float
accuracies = [float(acc) for acc in accuracies]

# Print accuracies
print("Accuracies:", accuracies)

# Print size of the array
print("Size of array:", len(accuracies))


Accuracies: [9.51, 9.51, 9.51, 9.59, 10.49, 12.03, 12.99, 13.97, 14.28, 14.7, 15.3, 16.12, 16.74, 17.27, 17.83, 18.36, 18.53, 18.87, 20.0, 21.7, 24.32, 24.86, 25.75, 26.95, 27.61, 28.1, 29.06, 29.38, 30.35, 30.09, 31.08, 31.84, 32.26, 31.86, 32.98, 33.22, 34.09, 34.77, 35.25, 35.55, 36.51, 37.46, 37.44, 37.59, 38.39, 38.59, 38.62, 39.0, 40.08, 39.9, 40.48, 40.51, 40.48, 40.91, 41.18, 41.38, 41.45, 42.17, 42.23, 42.01, 43.41, 42.65, 42.76, 43.47, 43.94, 43.97, 44.44, 44.25, 43.81, 44.55, 44.97, 45.03, 45.36, 45.93, 45.76, 45.61, 46.72, 46.76, 46.02, 46.23, 46.79, 47.67, 47.21, 47.32, 47.66, 48.43, 48.33, 48.17, 48.2, 47.38, 48.92, 49.72, 49.88, 50.11, 49.05, 49.04, 49.55, 50.44, 50.79, 50.36, 50.95, 51.36, 50.44, 51.21, 50.53, 51.4, 51.92, 50.25, 51.02, 51.68, 52.63, 52.26, 52.45, 52.38, 52.19, 53.0, 51.67, 52.86, 53.14, 52.92, 53.14, 53.21, 52.94, 53.15, 53.72, 53.89, 53.52, 53.97, 54.15, 54.06, 54.34, 54.74, 53.76, 54.03, 53.63, 54.41, 54.29, 54.23, 54.45, 53.7, 54.79, 55.2, 53.42, 55

In [ ]:
import numpy as np
from scipy.stats import truncnorm
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import CIFAR10
from typing import Dict, Tuple
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import random

# Define VAE model
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim, z_dim):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 4x4
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 2*z_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 2048),
            nn.ReLU(),
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, stride=2, padding=1),  # 32x32
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        mu, logvar = h[:, :self.z_dim], h[:, self.z_dim:]
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


# Define VAE training procedure
def vae_train(vae: VAE, trainloader: DataLoader, epochs: int) -> None:
    optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    for epoch in range(epochs):
        vae.train()
        for i, data in enumerate(trainloader, 0):
            inputs, _ = data
            optimizer.zero_grad()
            recon_x, mu, logvar = vae(inputs)
            loss = vae_loss(recon_x, inputs, mu, logvar)
            loss.backward()
            optimizer.step()

        scheduler.step()


# Define classification model
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load CIFAR10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

full_dataset = CIFAR10(root="./data", train=True, download=True, transform=transform)
test_set = CIFAR10(root="./data", train=False, download=True, transform=transform)

# Separate dataset by class
class_indices = {i: [] for i in range(10)}  # CIFAR-10 has 10 classes
for idx, (_, label) in enumerate(full_dataset):
    class_indices[label].append(idx)

# Define target count per class, summing to 60,000 with random distribution
class_counts = np.random.multinomial(60000, [0.1] * 10)  # Adjust probabilities if you want specific class biases
print("Random Images per Class:", class_counts)

# Sample indices based on the specified class counts
indices = []
for class_id, count in enumerate(class_counts):
    # Ensure count does not exceed available images
    count = min(count, len(class_indices[class_id]))
    selected_indices = random.sample(class_indices[class_id], count)
    indices.extend(selected_indices)

# Create a custom CIFAR-10 dataset with the sampled indices
custom_dataset = Subset(full_dataset, indices)
# Split dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create training and validation loaders
trainloader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(val_set, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)



# Define VAE loss function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Define training procedure for classification model
def train(net: nn.Module, trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

    best_acc = 0.0

    for epoch in range(epochs):
        # Training loop
        net.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        scheduler.step()

        # Validation loop
        net.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in valloader:
                images, labels = data
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(net.state_dict(), 'best_model.pth')

        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss / (i+1):.3f}, Validation Accuracy: {val_acc:.2f}%")

# Define evaluation procedure
def evaluate(net: nn.Module, testloader: DataLoader) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    net.eval()
    correct = 0
    total = 0
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_predictions)
    print(f"Confusion Matrix:\n{cm}")

    # Calculate TP, FP, TN, FN for each class
    tp = np.diag(cm)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (fp + fn + tp)

    # Calculate precision, recall, and F1 score for each class
    precision = precision_score(all_labels, all_predictions, average=None)
    recall = recall_score(all_labels, all_predictions, average=None)
    f1 = f1_score(all_labels, all_predictions, average=None)

    return accuracy, tp, fp, tn, fn, precision, recall, f1

# Initialize clients
def initialize_clients(trainset, transform, batch_size, num_clients):
    clients = {}
    for i in range(num_clients):
        client_trainset = torch.utils.data.Subset(trainset, range(i * len(trainset) // num_clients, (i + 1) * len(trainset) // num_clients))
        client_trainloader = torch.utils.data.DataLoader(client_trainset, batch_size=batch_size, shuffle=True)
        clients[f"client_{i}"] = client_trainloader
    return clients

def get_distribution_info(vae: VAE) -> Dict:
    # Implement the logic to extract distribution information from the VAE
    # This can involve computing statistics, parameters, or any other relevant information
    # that can be used to generate augmented data

    # Example implementation:
    distribution_info = {

        "normal": {
            "mean": vae.encoder[-1].bias.data.cpu().numpy(),
            "std": torch.exp(0.5 * vae.encoder[-1].weight.data).cpu().numpy()
        }
    }

    return distribution_info

def send_distribution_info(distribution_info: Dict) -> None:
    # Implement the logic to send the distribution information to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the information

    # Example implementation:
    # Send the distribution information to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

def generate_augmented_data(vae: VAE, distribution_info_normal: Dict) -> torch.Tensor:
    # Generate augmented data using both uniform and truncated uniform distributions



    mean_normal = distribution_info_normal["mean"]
    std_normal = distribution_info_normal["std"]

    augmented_data_normal = torch.randn(64, vae.z_dim) * std_normal + mean_normal





    # Calculate the average of augmented data from both distributions
    augmented_data_average =  augmented_data_normal

    return augmented_data_average

def federated_train(net: nn.Module, vae: VAE, trainloaders: Dict[str, DataLoader], trainloader: DataLoader, valloader: DataLoader, epochs: int) -> None:
    for epoch in range(epochs):
        for client_id, client_trainloader in trainloaders.items():
            # Train VAE on client data
            vae_train(vae, client_trainloader, epochs=10)

            # Share distribution information with global server
            distribution_info = get_distribution_info(vae)
            send_distribution_info(distribution_info)

            # Receive distribution information from other clients
            other_distribution_info = receive_distribution_info()

            # Generate augmented data using received distribution information
            augmented_data = generate_augmented_data(vae, other_distribution_info["normal"])

            # Train classification model using local, augmented, and validation data
            train(net, client_trainloader, valloader, epochs=10)

            # Send model updates to global server
            send_model_update(client_id, net.state_dict())

# Define logic to receive distribution information from global server
def receive_distribution_info() -> Dict:
    # Receive distribution information logic
    distribution_info = {

        "normal": {
            "mean": np.zeros(20),  # Adjust the size based on your latent space dimension
            "std": np.ones(20)
        }
    }
    return distribution_info

def send_model_update(client_id: str, model_update: Dict) -> None:
    # Implement the logic to send the model update to the global server
    # This can involve using a network protocol, a message queue, or any other communication mechanism
    # to send the model update

    # Example implementation:
    # Send the model update to the global server using a network protocol
    # For example, you can use the `socket` module to send the information over a network
    # or use a message queue like `RabbitMQ` to send the information
    pass

# Define global server procedure
def global_server() -> None:
    net = Net()
    x_dim = 3 * 32 * 32  # CIFAR-10 input size
    h_dim = 400
    z_dim = 20
    vae = VAE(x_dim, h_dim, z_dim)  # Initialize VAE object with required arguments

    # Initialize clients
    num_clients = 5  # Define the number of clients
    clients = initialize_clients(train_set, transform, batch_size=128, num_clients=num_clients)

    # Train model using FedDIS
    federated_train(net, vae, clients, trainloader, valloader, epochs=10)

    # Evaluate final model
    test_accuracy, tp, fp, tn, fn, precision, recall, f1 = evaluate(net, testloader)
    print(f"Test Accuracy: {test_accuracy:.2f}%")

    print("True Positives (TP):", tp)
    print("False Positives (FP):", fp)
    print("True Negatives (TN):", tn)
    print("False Negatives (FN):", fn)
    print(":", fn+tn+tp+fp)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

if __name__ == "__main__":
    global_server()

Files already downloaded and verified
Files already downloaded and verified
Random Images per Class: [5987 5897 5878 6031 6114 6065 6057 6028 5940 6003]
Epoch [1/10], Training Loss: 2.304, Validation Accuracy: 10.63%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.65%
Epoch [3/10], Training Loss: 2.301, Validation Accuracy: 12.53%
Epoch [4/10], Training Loss: 2.300, Validation Accuracy: 15.01%
Epoch [5/10], Training Loss: 2.298, Validation Accuracy: 15.84%
Epoch [6/10], Training Loss: 2.296, Validation Accuracy: 16.61%
Epoch [7/10], Training Loss: 2.292, Validation Accuracy: 17.22%
Epoch [8/10], Training Loss: 2.288, Validation Accuracy: 17.73%
Epoch [9/10], Training Loss: 2.281, Validation Accuracy: 18.03%
Epoch [10/10], Training Loss: 2.271, Validation Accuracy: 18.24%
Epoch [1/10], Training Loss: 2.256, Validation Accuracy: 18.29%
Epoch [2/10], Training Loss: 2.232, Validation Accuracy: 18.64%
Epoch [3/10], Training Loss: 2.199, Validation Accuracy: 19.14%
Epoch [4/10], 

In [ ]:
import re

# Your provided text
log = """
Epoch [1/10], Training Loss: 2.304, Validation Accuracy: 10.63%
Epoch [2/10], Training Loss: 2.303, Validation Accuracy: 10.65%
Epoch [3/10], Training Loss: 2.301, Validation Accuracy: 12.53%
Epoch [4/10], Training Loss: 2.300, Validation Accuracy: 15.01%
Epoch [5/10], Training Loss: 2.298, Validation Accuracy: 15.84%
Epoch [6/10], Training Loss: 2.296, Validation Accuracy: 16.61%
Epoch [7/10], Training Loss: 2.292, Validation Accuracy: 17.22%
Epoch [8/10], Training Loss: 2.288, Validation Accuracy: 17.73%
Epoch [9/10], Training Loss: 2.281, Validation Accuracy: 18.03%
Epoch [10/10], Training Loss: 2.271, Validation Accuracy: 18.24%
Epoch [1/10], Training Loss: 2.256, Validation Accuracy: 18.29%
Epoch [2/10], Training Loss: 2.232, Validation Accuracy: 18.64%
Epoch [3/10], Training Loss: 2.199, Validation Accuracy: 19.14%
Epoch [4/10], Training Loss: 2.165, Validation Accuracy: 20.29%
Epoch [5/10], Training Loss: 2.134, Validation Accuracy: 22.04%
Epoch [6/10], Training Loss: 2.106, Validation Accuracy: 23.48%
Epoch [7/10], Training Loss: 2.081, Validation Accuracy: 24.06%
Epoch [8/10], Training Loss: 2.058, Validation Accuracy: 24.25%
Epoch [9/10], Training Loss: 2.040, Validation Accuracy: 25.54%
Epoch [10/10], Training Loss: 2.025, Validation Accuracy: 25.73%
Epoch [1/10], Training Loss: 2.022, Validation Accuracy: 26.48%
Epoch [2/10], Training Loss: 2.008, Validation Accuracy: 27.31%
Epoch [3/10], Training Loss: 1.993, Validation Accuracy: 27.95%
Epoch [4/10], Training Loss: 1.981, Validation Accuracy: 27.97%
Epoch [5/10], Training Loss: 1.968, Validation Accuracy: 28.70%
Epoch [6/10], Training Loss: 1.955, Validation Accuracy: 29.93%
Epoch [7/10], Training Loss: 1.942, Validation Accuracy: 30.11%
Epoch [8/10], Training Loss: 1.930, Validation Accuracy: 30.36%
Epoch [9/10], Training Loss: 1.915, Validation Accuracy: 31.10%
Epoch [10/10], Training Loss: 1.901, Validation Accuracy: 31.93%
Epoch [1/10], Training Loss: 1.876, Validation Accuracy: 32.49%
Epoch [2/10], Training Loss: 1.856, Validation Accuracy: 33.43%
Epoch [3/10], Training Loss: 1.841, Validation Accuracy: 33.96%
Epoch [4/10], Training Loss: 1.819, Validation Accuracy: 34.24%
Epoch [5/10], Training Loss: 1.800, Validation Accuracy: 35.30%
Epoch [6/10], Training Loss: 1.779, Validation Accuracy: 36.12%
Epoch [7/10], Training Loss: 1.759, Validation Accuracy: 35.80%
Epoch [8/10], Training Loss: 1.738, Validation Accuracy: 36.71%
Epoch [9/10], Training Loss: 1.720, Validation Accuracy: 37.00%
Epoch [10/10], Training Loss: 1.701, Validation Accuracy: 37.50%
Epoch [1/10], Training Loss: 1.710, Validation Accuracy: 37.77%
Epoch [2/10], Training Loss: 1.693, Validation Accuracy: 38.55%
Epoch [3/10], Training Loss: 1.676, Validation Accuracy: 38.23%
Epoch [4/10], Training Loss: 1.663, Validation Accuracy: 39.57%
Epoch [5/10], Training Loss: 1.651, Validation Accuracy: 39.43%
Epoch [6/10], Training Loss: 1.638, Validation Accuracy: 40.00%
Epoch [7/10], Training Loss: 1.626, Validation Accuracy: 40.53%
Epoch [8/10], Training Loss: 1.619, Validation Accuracy: 40.64%
Epoch [9/10], Training Loss: 1.609, Validation Accuracy: 41.21%
Epoch [10/10], Training Loss: 1.594, Validation Accuracy: 41.65%
Epoch [1/10], Training Loss: 1.632, Validation Accuracy: 41.89%
Epoch [2/10], Training Loss: 1.619, Validation Accuracy: 41.41%
Epoch [3/10], Training Loss: 1.600, Validation Accuracy: 42.37%
Epoch [4/10], Training Loss: 1.598, Validation Accuracy: 42.68%
Epoch [5/10], Training Loss: 1.588, Validation Accuracy: 42.57%
Epoch [6/10], Training Loss: 1.572, Validation Accuracy: 43.18%
Epoch [7/10], Training Loss: 1.568, Validation Accuracy: 43.22%
Epoch [8/10], Training Loss: 1.559, Validation Accuracy: 43.70%
Epoch [9/10], Training Loss: 1.548, Validation Accuracy: 44.55%
Epoch [10/10], Training Loss: 1.545, Validation Accuracy: 44.10%
Epoch [1/10], Training Loss: 1.560, Validation Accuracy: 44.37%
Epoch [2/10], Training Loss: 1.543, Validation Accuracy: 45.17%
Epoch [3/10], Training Loss: 1.537, Validation Accuracy: 44.57%
Epoch [4/10], Training Loss: 1.524, Validation Accuracy: 45.02%
Epoch [5/10], Training Loss: 1.515, Validation Accuracy: 44.36%
Epoch [6/10], Training Loss: 1.507, Validation Accuracy: 44.68%
Epoch [7/10], Training Loss: 1.496, Validation Accuracy: 45.39%
Epoch [8/10], Training Loss: 1.491, Validation Accuracy: 45.88%
Epoch [9/10], Training Loss: 1.489, Validation Accuracy: 44.94%
Epoch [10/10], Training Loss: 1.482, Validation Accuracy: 46.33%
Epoch [1/10], Training Loss: 1.499, Validation Accuracy: 46.75%
Epoch [2/10], Training Loss: 1.488, Validation Accuracy: 46.79%
Epoch [3/10], Training Loss: 1.476, Validation Accuracy: 47.10%
Epoch [4/10], Training Loss: 1.470, Validation Accuracy: 46.53%
Epoch [5/10], Training Loss: 1.455, Validation Accuracy: 46.49%
Epoch [6/10], Training Loss: 1.451, Validation Accuracy: 46.99%
Epoch [7/10], Training Loss: 1.443, Validation Accuracy: 47.39%
Epoch [8/10], Training Loss: 1.430, Validation Accuracy: 46.19%
Epoch [9/10], Training Loss: 1.436, Validation Accuracy: 47.79%
Epoch [10/10], Training Loss: 1.419, Validation Accuracy: 47.39%
Epoch [1/10], Training Loss: 1.451, Validation Accuracy: 47.94%
Epoch [2/10], Training Loss: 1.437, Validation Accuracy: 47.87%
Epoch [3/10], Training Loss: 1.425, Validation Accuracy: 48.20%
Epoch [4/10], Training Loss: 1.422, Validation Accuracy: 48.09%
Epoch [5/10], Training Loss: 1.404, Validation Accuracy: 48.60%
Epoch [6/10], Training Loss: 1.395, Validation Accuracy: 48.58%
Epoch [7/10], Training Loss: 1.391, Validation Accuracy: 48.44%
Epoch [8/10], Training Loss: 1.380, Validation Accuracy: 49.38%
Epoch [9/10], Training Loss: 1.379, Validation Accuracy: 49.27%
Epoch [10/10], Training Loss: 1.372, Validation Accuracy: 48.94%
Epoch [1/10], Training Loss: 1.403, Validation Accuracy: 49.25%
Epoch [2/10], Training Loss: 1.378, Validation Accuracy: 50.23%
Epoch [3/10], Training Loss: 1.367, Validation Accuracy: 49.71%
Epoch [4/10], Training Loss: 1.359, Validation Accuracy: 50.06%
Epoch [5/10], Training Loss: 1.355, Validation Accuracy: 50.22%
Epoch [6/10], Training Loss: 1.342, Validation Accuracy: 50.55%
Epoch [7/10], Training Loss: 1.336, Validation Accuracy: 50.25%
Epoch [8/10], Training Loss: 1.325, Validation Accuracy: 50.75%
Epoch [9/10], Training Loss: 1.315, Validation Accuracy: 50.64%
Epoch [10/10], Training Loss: 1.309, Validation Accuracy: 51.10%
Epoch [1/10], Training Loss: 1.387, Validation Accuracy: 50.93%
Epoch [2/10], Training Loss: 1.370, Validation Accuracy: 49.48%
Epoch [3/10], Training Loss: 1.366, Validation Accuracy: 51.56%
Epoch [4/10], Training Loss: 1.344, Validation Accuracy: 51.36%
Epoch [5/10], Training Loss: 1.336, Validation Accuracy: 51.61%
Epoch [6/10], Training Loss: 1.326, Validation Accuracy: 51.57%
Epoch [7/10], Training Loss: 1.318, Validation Accuracy: 51.84%
Epoch [8/10], Training Loss: 1.306, Validation Accuracy: 50.76%
Epoch [9/10], Training Loss: 1.306, Validation Accuracy: 51.94%
Epoch [10/10], Training Loss: 1.298, Validation Accuracy: 52.48%
Epoch [1/10], Training Loss: 1.349, Validation Accuracy: 51.30%
Epoch [2/10], Training Loss: 1.335, Validation Accuracy: 51.24%
Epoch [3/10], Training Loss: 1.315, Validation Accuracy: 52.84%
Epoch [4/10], Training Loss: 1.308, Validation Accuracy: 52.46%
Epoch [5/10], Training Loss: 1.302, Validation Accuracy: 52.85%
Epoch [6/10], Training Loss: 1.287, Validation Accuracy: 52.53%
Epoch [7/10], Training Loss: 1.283, Validation Accuracy: 52.29%
Epoch [8/10], Training Loss: 1.274, Validation Accuracy: 52.66%
Epoch [9/10], Training Loss: 1.262, Validation Accuracy: 52.52%
Epoch [10/10], Training Loss: 1.254, Validation Accuracy: 52.70%
Epoch [1/10], Training Loss: 1.310, Validation Accuracy: 52.43%
Epoch [2/10], Training Loss: 1.289, Validation Accuracy: 53.74%
Epoch [3/10], Training Loss: 1.274, Validation Accuracy: 53.61%
Epoch [4/10], Training Loss: 1.257, Validation Accuracy: 53.54%
Epoch [5/10], Training Loss: 1.251, Validation Accuracy: 52.69%
Epoch [6/10], Training Loss: 1.249, Validation Accuracy: 53.28%
Epoch [7/10], Training Loss: 1.230, Validation Accuracy: 52.36%
Epoch [8/10], Training Loss: 1.229, Validation Accuracy: 51.92%
Epoch [9/10], Training Loss: 1.222, Validation Accuracy: 53.71%
Epoch [10/10], Training Loss: 1.207, Validation Accuracy: 54.17%
Epoch [1/10], Training Loss: 1.285, Validation Accuracy: 52.43%
Epoch [2/10], Training Loss: 1.265, Validation Accuracy: 52.70%
Epoch [3/10], Training Loss: 1.246, Validation Accuracy: 54.28%
Epoch [4/10], Training Loss: 1.238, Validation Accuracy: 54.36%
Epoch [5/10], Training Loss: 1.222, Validation Accuracy: 54.34%
Epoch [6/10], Training Loss: 1.213, Validation Accuracy: 54.22%
Epoch [7/10], Training Loss: 1.215, Validation Accuracy: 54.97%
Epoch [8/10], Training Loss: 1.196, Validation Accuracy: 54.17%
Epoch [9/10], Training Loss: 1.194, Validation Accuracy: 54.32%
Epoch [10/10], Training Loss: 1.184, Validation Accuracy: 55.15%
Epoch [1/10], Training Loss: 1.227, Validation Accuracy: 54.46%
Epoch [2/10], Training Loss: 1.207, Validation Accuracy: 55.00%
Epoch [3/10], Training Loss: 1.190, Validation Accuracy: 54.94%
Epoch [4/10], Training Loss: 1.180, Validation Accuracy: 55.22%
Epoch [5/10], Training Loss: 1.171, Validation Accuracy: 56.01%
Epoch [6/10], Training Loss: 1.169, Validation Accuracy: 54.93%
Epoch [7/10], Training Loss: 1.164, Validation Accuracy: 54.98%
Epoch [8/10], Training Loss: 1.143, Validation Accuracy: 55.47%
Epoch [9/10], Training Loss: 1.132, Validation Accuracy: 55.57%
Epoch [10/10], Training Loss: 1.128, Validation Accuracy: 56.00%
Epoch [1/10], Training Loss: 1.228, Validation Accuracy: 55.98%
Epoch [2/10], Training Loss: 1.197, Validation Accuracy: 56.21%
Epoch [3/10], Training Loss: 1.180, Validation Accuracy: 55.72%
Epoch [4/10], Training Loss: 1.172, Validation Accuracy: 56.28%
Epoch [5/10], Training Loss: 1.164, Validation Accuracy: 56.37%
Epoch [6/10], Training Loss: 1.152, Validation Accuracy: 55.90%
Epoch [7/10], Training Loss: 1.142, Validation Accuracy: 56.08%
Epoch [8/10], Training Loss: 1.138, Validation Accuracy: 55.32%
Epoch [9/10], Training Loss: 1.124, Validation Accuracy: 56.73%
Epoch [10/10], Training Loss: 1.115, Validation Accuracy: 56.45%
Epoch [1/10], Training Loss: 1.209, Validation Accuracy: 56.68%
Epoch [2/10], Training Loss: 1.187, Validation Accuracy: 57.28%
Epoch [3/10], Training Loss: 1.171, Validation Accuracy: 56.49%
Epoch [4/10], Training Loss: 1.173, Validation Accuracy: 56.33%
Epoch [5/10], Training Loss: 1.148, Validation Accuracy: 57.36%
Epoch [6/10], Training Loss: 1.133, Validation Accuracy: 55.98%
Epoch [7/10], Training Loss: 1.129, Validation Accuracy: 56.85%
Epoch [8/10], Training Loss: 1.114, Validation Accuracy: 56.79%
Epoch [9/10], Training Loss: 1.100, Validation Accuracy: 57.22%
Epoch [10/10], Training Loss: 1.102, Validation Accuracy: 56.48%
Epoch [1/10], Training Loss: 1.182, Validation Accuracy: 57.33%
Epoch [2/10], Training Loss: 1.151, Validation Accuracy: 57.68%
Epoch [3/10], Training Loss: 1.137, Validation Accuracy: 57.91%
Epoch [4/10], Training Loss: 1.119, Validation Accuracy: 57.68%
Epoch [5/10], Training Loss: 1.107, Validation Accuracy: 57.98%
Epoch [6/10], Training Loss: 1.094, Validation Accuracy: 57.94%
Epoch [7/10], Training Loss: 1.084, Validation Accuracy: 57.55%
Epoch [8/10], Training Loss: 1.076, Validation Accuracy: 57.62%
Epoch [9/10], Training Loss: 1.073, Validation Accuracy: 56.00%
Epoch [10/10], Training Loss: 1.073, Validation Accuracy: 57.69%
Epoch [1/10], Training Loss: 1.169, Validation Accuracy: 57.75%
Epoch [2/10], Training Loss: 1.141, Validation Accuracy: 58.10%
Epoch [3/10], Training Loss: 1.116, Validation Accuracy: 57.15%
Epoch [4/10], Training Loss: 1.107, Validation Accuracy: 57.99%
Epoch [5/10], Training Loss: 1.096, Validation Accuracy: 58.34%
Epoch [6/10], Training Loss: 1.083, Validation Accuracy: 58.27%
Epoch [7/10], Training Loss: 1.067, Validation Accuracy: 57.81%
Epoch [8/10], Training Loss: 1.059, Validation Accuracy: 58.32%
Epoch [9/10], Training Loss: 1.054, Validation Accuracy: 58.00%
Epoch [10/10], Training Loss: 1.044, Validation Accuracy: 57.47%
Epoch [1/10], Training Loss: 1.115, Validation Accuracy: 58.75%
Epoch [2/10], Training Loss: 1.086, Validation Accuracy: 58.98%
Epoch [3/10], Training Loss: 1.071, Validation Accuracy: 59.21%
Epoch [4/10], Training Loss: 1.056, Validation Accuracy: 58.97%
Epoch [5/10], Training Loss: 1.042, Validation Accuracy: 58.33%
Epoch [6/10], Training Loss: 1.032, Validation Accuracy: 58.78%
Epoch [7/10], Training Loss: 1.019, Validation Accuracy: 58.66%
Epoch [8/10], Training Loss: 1.006, Validation Accuracy: 58.97%
Epoch [9/10], Training Loss: 1.000, Validation Accuracy: 57.36%
Epoch [10/10], Training Loss: 0.998, Validation Accuracy: 57.88%
Epoch [1/10], Training Loss: 1.117, Validation Accuracy: 59.28%
Epoch [2/10], Training Loss: 1.080, Validation Accuracy: 58.01%
Epoch [3/10], Training Loss: 1.074, Validation Accuracy: 59.07%
Epoch [4/10], Training Loss: 1.048, Validation Accuracy: 59.23%
Epoch [5/10], Training Loss: 1.033, Validation Accuracy: 59.39%
Epoch [6/10], Training Loss: 1.016, Validation Accuracy: 59.44%
Epoch [7/10], Training Loss: 1.011, Validation Accuracy: 59.20%
Epoch [8/10], Training Loss: 0.994, Validation Accuracy: 59.21%
Epoch [9/10], Training Loss: 0.981, Validation Accuracy: 59.03%
Epoch [10/10], Training Loss: 0.979, Validation Accuracy: 59.34%
Epoch [1/10], Training Loss: 1.119, Validation Accuracy: 59.81%
Epoch [2/10], Training Loss: 1.079, Validation Accuracy: 58.81%
Epoch [3/10], Training Loss: 1.061, Validation Accuracy: 59.75%
Epoch [4/10], Training Loss: 1.045, Validation Accuracy: 59.39%
Epoch [5/10], Training Loss: 1.027, Validation Accuracy: 59.81%
Epoch [6/10], Training Loss: 1.024, Validation Accuracy: 59.56%
Epoch [7/10], Training Loss: 1.004, Validation Accuracy: 59.60%
Epoch [8/10], Training Loss: 0.987, Validation Accuracy: 59.85%
Epoch [9/10], Training Loss: 0.985, Validation Accuracy: 59.78%
Epoch [10/10], Training Loss: 0.963, Validation Accuracy: 59.50%
Epoch [1/10], Training Loss: 1.086, Validation Accuracy: 59.98%
Epoch [2/10], Training Loss: 1.050, Validation Accuracy: 59.18%
Epoch [3/10], Training Loss: 1.031, Validation Accuracy: 60.33%
Epoch [4/10], Training Loss: 1.018, Validation Accuracy: 60.49%
Epoch [5/10], Training Loss: 0.994, Validation Accuracy: 59.89%
Epoch [6/10], Training Loss: 0.982, Validation Accuracy: 59.23%
Epoch [7/10], Training Loss: 0.970, Validation Accuracy: 60.72%
Epoch [8/10], Training Loss: 0.954, Validation Accuracy: 59.97%
Epoch [9/10], Training Loss: 0.941, Validation Accuracy: 59.01%
Epoch [10/10], Training Loss: 0.927, Validation Accuracy: 60.00%
Epoch [1/10], Training Loss: 1.080, Validation Accuracy: 59.98%
Epoch [2/10], Training Loss: 1.044, Validation Accuracy: 60.24%
Epoch [3/10], Training Loss: 1.019, Validation Accuracy: 60.55%
Epoch [4/10], Training Loss: 0.989, Validation Accuracy: 60.77%
Epoch [5/10], Training Loss: 0.982, Validation Accuracy: 59.94%
Epoch [6/10], Training Loss: 0.968, Validation Accuracy: 60.14%
Epoch [7/10], Training Loss: 0.961, Validation Accuracy: 59.85%
Epoch [8/10], Training Loss: 0.946, Validation Accuracy: 58.95%
Epoch [9/10], Training Loss: 0.927, Validation Accuracy: 59.42%
Epoch [10/10], Training Loss: 0.916, Validation Accuracy: 60.84%
Epoch [1/10], Training Loss: 1.042, Validation Accuracy: 59.78%
Epoch [2/10], Training Loss: 0.999, Validation Accuracy: 60.12%
Epoch [3/10], Training Loss: 0.979, Validation Accuracy: 60.28%
Epoch [4/10], Training Loss: 0.954, Validation Accuracy: 60.79%
Epoch [5/10], Training Loss: 0.940, Validation Accuracy: 60.07%
Epoch [6/10], Training Loss: 0.926, Validation Accuracy: 60.27%
Epoch [7/10], Training Loss: 0.913, Validation Accuracy: 59.44%
Epoch [8/10], Training Loss: 0.897, Validation Accuracy: 61.15%
Epoch [9/10], Training Loss: 0.877, Validation Accuracy: 60.50%
Epoch [10/10], Training Loss: 0.876, Validation Accuracy: 60.23%
Epoch [1/10], Training Loss: 1.040, Validation Accuracy: 60.70%
Epoch [2/10], Training Loss: 0.994, Validation Accuracy: 60.94%
Epoch [3/10], Training Loss: 0.972, Validation Accuracy: 61.05%
Epoch [4/10], Training Loss: 0.947, Validation Accuracy: 60.53%
Epoch [5/10], Training Loss: 0.932, Validation Accuracy: 61.31%
Epoch [6/10], Training Loss: 0.914, Validation Accuracy: 60.56%
Epoch [7/10], Training Loss: 0.899, Validation Accuracy: 60.94%
Epoch [8/10], Training Loss: 0.882, Validation Accuracy: 60.89%
Epoch [9/10], Training Loss: 0.873, Validation Accuracy: 59.55%
Epoch [10/10], Training Loss: 0.869, Validation Accuracy: 60.91%
Epoch [1/10], Training Loss: 1.046, Validation Accuracy: 61.31%
Epoch [2/10], Training Loss: 1.004, Validation Accuracy: 61.77%
Epoch [3/10], Training Loss: 0.975, Validation Accuracy: 61.61%
Epoch [4/10], Training Loss: 0.950, Validation Accuracy: 61.15%
Epoch [5/10], Training Loss: 0.922, Validation Accuracy: 61.35%
Epoch [6/10], Training Loss: 0.912, Validation Accuracy: 60.90%
Epoch [7/10], Training Loss: 0.894, Validation Accuracy: 61.88%
Epoch [8/10], Training Loss: 0.874, Validation Accuracy: 61.71%
Epoch [9/10], Training Loss: 0.865, Validation Accuracy: 61.25%
Epoch [10/10], Training Loss: 0.846, Validation Accuracy: 61.17%
Epoch [1/10], Training Loss: 1.026, Validation Accuracy: 61.36%
Epoch [2/10], Training Loss: 0.974, Validation Accuracy: 61.74%
Epoch [3/10], Training Loss: 0.930, Validation Accuracy: 60.86%
Epoch [4/10], Training Loss: 0.913, Validation Accuracy: 61.20%
Epoch [5/10], Training Loss: 0.907, Validation Accuracy: 60.93%
Epoch [6/10], Training Loss: 0.882, Validation Accuracy: 61.64%
Epoch [7/10], Training Loss: 0.873, Validation Accuracy: 61.39%
Epoch [8/10], Training Loss: 0.849, Validation Accuracy: 61.24%
Epoch [9/10], Training Loss: 0.828, Validation Accuracy: 61.33%
Epoch [10/10], Training Loss: 0.817, Validation Accuracy: 61.02%
Epoch [1/10], Training Loss: 1.018, Validation Accuracy: 61.08%
Epoch [2/10], Training Loss: 0.961, Validation Accuracy: 61.25%
Epoch [3/10], Training Loss: 0.934, Validation Accuracy: 61.20%
Epoch [4/10], Training Loss: 0.913, Validation Accuracy: 61.89%
Epoch [5/10], Training Loss: 0.881, Validation Accuracy: 61.66%
Epoch [6/10], Training Loss: 0.869, Validation Accuracy: 61.63%
Epoch [7/10], Training Loss: 0.852, Validation Accuracy: 61.39%
Epoch [8/10], Training Loss: 0.838, Validation Accuracy: 61.56%
Epoch [9/10], Training Loss: 0.831, Validation Accuracy: 61.28%
Epoch [10/10], Training Loss: 0.809, Validation Accuracy: 61.71%
Epoch [1/10], Training Loss: 0.975, Validation Accuracy: 61.37%
Epoch [2/10], Training Loss: 0.933, Validation Accuracy: 62.03%
Epoch [3/10], Training Loss: 0.893, Validation Accuracy: 62.50%
Epoch [4/10], Training Loss: 0.861, Validation Accuracy: 62.30%
Epoch [5/10], Training Loss: 0.846, Validation Accuracy: 61.76%
Epoch [6/10], Training Loss: 0.824, Validation Accuracy: 61.54%
Epoch [7/10], Training Loss: 0.813, Validation Accuracy: 61.39%
Epoch [8/10], Training Loss: 0.797, Validation Accuracy: 61.42%
Epoch [9/10], Training Loss: 0.779, Validation Accuracy: 61.78%
Epoch [10/10], Training Loss: 0.772, Validation Accuracy: 62.17%
Epoch [1/10], Training Loss: 0.962, Validation Accuracy: 62.01%
Epoch [2/10], Training Loss: 0.909, Validation Accuracy: 62.07%
Epoch [3/10], Training Loss: 0.881, Validation Accuracy: 61.96%
Epoch [4/10], Training Loss: 0.852, Validation Accuracy: 62.36%
Epoch [5/10], Training Loss: 0.830, Validation Accuracy: 62.06%
Epoch [6/10], Training Loss: 0.814, Validation Accuracy: 60.31%
Epoch [7/10], Training Loss: 0.802, Validation Accuracy: 61.94%
Epoch [8/10], Training Loss: 0.783, Validation Accuracy: 61.65%
Epoch [9/10], Training Loss: 0.760, Validation Accuracy: 61.27%
Epoch [10/10], Training Loss: 0.761, Validation Accuracy: 61.31%
Epoch [1/10], Training Loss: 0.984, Validation Accuracy: 62.82%
Epoch [2/10], Training Loss: 0.913, Validation Accuracy: 61.88%
Epoch [3/10], Training Loss: 0.875, Validation Accuracy: 62.51%
Epoch [4/10], Training Loss: 0.856, Validation Accuracy: 61.77%
Epoch [5/10], Training Loss: 0.833, Validation Accuracy: 61.87%
Epoch [6/10], Training Loss: 0.806, Validation Accuracy: 62.11%
Epoch [7/10], Training Loss: 0.794, Validation Accuracy: 62.11%
Epoch [8/10], Training Loss: 0.777, Validation Accuracy: 62.24%
Epoch [9/10], Training Loss: 0.761, Validation Accuracy: 61.71%
Epoch [10/10], Training Loss: 0.747, Validation Accuracy: 61.55%
Epoch [1/10], Training Loss: 0.964, Validation Accuracy: 61.15%
Epoch [2/10], Training Loss: 0.900, Validation Accuracy: 61.98%
Epoch [3/10], Training Loss: 0.868, Validation Accuracy: 62.19%
Epoch [4/10], Training Loss: 0.830, Validation Accuracy: 61.46%
Epoch [5/10], Training Loss: 0.801, Validation Accuracy: 61.92%
Epoch [6/10], Training Loss: 0.780, Validation Accuracy: 62.14%
Epoch [7/10], Training Loss: 0.771, Validation Accuracy: 61.97%
Epoch [8/10], Training Loss: 0.757, Validation Accuracy: 62.40%
Epoch [9/10], Training Loss: 0.733, Validation Accuracy: 61.76%
Epoch [10/10], Training Loss: 0.728, Validation Accuracy: 61.65%
Epoch [1/10], Training Loss: 0.957, Validation Accuracy: 61.02%
Epoch [2/10], Training Loss: 0.894, Validation Accuracy: 62.18%
Epoch [3/10], Training Loss: 0.851, Validation Accuracy: 62.01%
Epoch [4/10], Training Loss: 0.836, Validation Accuracy: 61.51%
Epoch [5/10], Training Loss: 0.803, Validation Accuracy: 61.93%
Epoch [6/10], Training Loss: 0.778, Validation Accuracy: 62.08%
Epoch [7/10], Training Loss: 0.764, Validation Accuracy: 61.96%
Epoch [8/10], Training Loss: 0.748, Validation Accuracy: 62.61%
Epoch [9/10], Training Loss: 0.732, Validation Accuracy: 62.16%
Epoch [10/10], Training Loss: 0.716, Validation Accuracy: 62.02%
Epoch [1/10], Training Loss: 0.926, Validation Accuracy: 62.14%
Epoch [2/10], Training Loss: 0.853, Validation Accuracy: 62.81%
Epoch [3/10], Training Loss: 0.821, Validation Accuracy: 62.08%
Epoch [4/10], Training Loss: 0.785, Validation Accuracy: 62.62%
Epoch [5/10], Training Loss: 0.757, Validation Accuracy: 62.62%
Epoch [6/10], Training Loss: 0.732, Validation Accuracy: 62.68%
Epoch [7/10], Training Loss: 0.715, Validation Accuracy: 62.16%
Epoch [8/10], Training Loss: 0.704, Validation Accuracy: 62.47%
Epoch [9/10], Training Loss: 0.680, Validation Accuracy: 61.38%
Epoch [10/10], Training Loss: 0.666, Validation Accuracy: 62.46%
Epoch [1/10], Training Loss: 0.901, Validation Accuracy: 62.48%
Epoch [2/10], Training Loss: 0.840, Validation Accuracy: 62.62%
Epoch [3/10], Training Loss: 0.804, Validation Accuracy: 61.85%
Epoch [4/10], Training Loss: 0.769, Validation Accuracy: 62.37%
Epoch [5/10], Training Loss: 0.742, Validation Accuracy: 62.50%
Epoch [6/10], Training Loss: 0.716, Validation Accuracy: 61.66%
Epoch [7/10], Training Loss: 0.698, Validation Accuracy: 62.00%
Epoch [8/10], Training Loss: 0.691, Validation Accuracy: 62.00%
Epoch [9/10], Training Loss: 0.671, Validation Accuracy: 61.97%
Epoch [10/10], Training Loss: 0.653, Validation Accuracy: 61.91%
Epoch [1/10], Training Loss: 0.933, Validation Accuracy: 59.38%
Epoch [2/10], Training Loss: 0.844, Validation Accuracy: 62.10%
Epoch [3/10], Training Loss: 0.795, Validation Accuracy: 62.88%
Epoch [4/10], Training Loss: 0.767, Validation Accuracy: 61.25%
Epoch [5/10], Training Loss: 0.751, Validation Accuracy: 62.06%
Epoch [6/10], Training Loss: 0.720, Validation Accuracy: 62.89%
Epoch [7/10], Training Loss: 0.691, Validation Accuracy: 62.56%
Epoch [8/10], Training Loss: 0.679, Validation Accuracy: 62.20%
Epoch [9/10], Training Loss: 0.664, Validation Accuracy: 61.96%
Epoch [10/10], Training Loss: 0.640, Validation Accuracy: 61.56%
Epoch [1/10], Training Loss: 0.920, Validation Accuracy: 62.30%
Epoch [2/10], Training Loss: 0.841, Validation Accuracy: 61.76%
Epoch [3/10], Training Loss: 0.784, Validation Accuracy: 62.60%
Epoch [4/10], Training Loss: 0.744, Validation Accuracy: 62.27%
Epoch [5/10], Training Loss: 0.725, Validation Accuracy: 62.12%
Epoch [6/10], Training Loss: 0.705, Validation Accuracy: 62.26%
Epoch [7/10], Training Loss: 0.676, Validation Accuracy: 62.73%
Epoch [8/10], Training Loss: 0.657, Validation Accuracy: 62.36%
Epoch [9/10], Training Loss: 0.637, Validation Accuracy: 62.41%
Epoch [10/10], Training Loss: 0.626, Validation Accuracy: 60.60%
Epoch [1/10], Training Loss: 0.914, Validation Accuracy: 62.22%
Epoch [2/10], Training Loss: 0.819, Validation Accuracy: 61.65%
Epoch [3/10], Training Loss: 0.782, Validation Accuracy: 62.38%
Epoch [4/10], Training Loss: 0.744, Validation Accuracy: 62.31%
Epoch [5/10], Training Loss: 0.723, Validation Accuracy: 60.94%
Epoch [6/10], Training Loss: 0.696, Validation Accuracy: 62.36%
Epoch [7/10], Training Loss: 0.672, Validation Accuracy: 61.94%
Epoch [8/10], Training Loss: 0.651, Validation Accuracy: 61.31%
Epoch [9/10], Training Loss: 0.636, Validation Accuracy: 62.41%
Epoch [10/10], Training Loss: 0.613, Validation Accuracy: 62.07%
Epoch [1/10], Training Loss: 0.879, Validation Accuracy: 62.84%
Epoch [2/10], Training Loss: 0.779, Validation Accuracy: 62.94%
Epoch [3/10], Training Loss: 0.739, Validation Accuracy: 62.57%
Epoch [4/10], Training Loss: 0.707, Validation Accuracy: 62.28%
Epoch [5/10], Training Loss: 0.673, Validation Accuracy: 62.85%
Epoch [6/10], Training Loss: 0.647, Validation Accuracy: 62.39%
Epoch [7/10], Training Loss: 0.624, Validation Accuracy: 62.36%
Epoch [8/10], Training Loss: 0.616, Validation Accuracy: 61.94%
Epoch [9/10], Training Loss: 0.587, Validation Accuracy: 61.37%
Epoch [10/10], Training Loss: 0.568, Validation Accuracy: 61.22%
Epoch [1/10], Training Loss: 0.845, Validation Accuracy: 62.47%
Epoch [2/10], Training Loss: 0.778, Validation Accuracy: 62.24%
Epoch [3/10], Training Loss: 0.722, Validation Accuracy: 62.19%
Epoch [4/10], Training Loss: 0.689, Validation Accuracy: 62.25%
Epoch [5/10], Training Loss: 0.660, Validation Accuracy: 61.55%
Epoch [6/10], Training Loss: 0.640, Validation Accuracy: 62.41%
Epoch [7/10], Training Loss: 0.607, Validation Accuracy: 62.02%
Epoch [8/10], Training Loss: 0.583, Validation Accuracy: 61.86%
Epoch [9/10], Training Loss: 0.568, Validation Accuracy: 61.82%
Epoch [10/10], Training Loss: 0.543, Validation Accuracy: 62.03%
Epoch [1/10], Training Loss: 0.866, Validation Accuracy: 61.28%
Epoch [2/10], Training Loss: 0.772, Validation Accuracy: 62.00%
Epoch [3/10], Training Loss: 0.725, Validation Accuracy: 62.71%
Epoch [4/10], Training Loss: 0.674, Validation Accuracy: 62.20%
Epoch [5/10], Training Loss: 0.655, Validation Accuracy: 62.34%
Epoch [6/10], Training Loss: 0.624, Validation Accuracy: 61.64%
Epoch [7/10], Training Loss: 0.606, Validation Accuracy: 62.33%
Epoch [8/10], Training Loss: 0.578, Validation Accuracy: 61.62%
Epoch [9/10], Training Loss: 0.555, Validation Accuracy: 61.96%
Epoch [10/10], Training Loss: 0.535, Validation Accuracy: 61.76%
Epoch [1/10], Training Loss: 0.867, Validation Accuracy: 61.26%
Epoch [2/10], Training Loss: 0.764, Validation Accuracy: 61.98%
Epoch [3/10], Training Loss: 0.715, Validation Accuracy: 61.67%
Epoch [4/10], Training Loss: 0.669, Validation Accuracy: 62.42%
Epoch [5/10], Training Loss: 0.646, Validation Accuracy: 62.57%
Epoch [6/10], Training Loss: 0.609, Validation Accuracy: 62.40%
Epoch [7/10], Training Loss: 0.592, Validation Accuracy: 62.25%
Epoch [8/10], Training Loss: 0.571, Validation Accuracy: 61.84%
Epoch [9/10], Training Loss: 0.544, Validation Accuracy: 61.90%
Epoch [10/10], Training Loss: 0.532, Validation Accuracy: 62.21%
Epoch [1/10], Training Loss: 0.889, Validation Accuracy: 61.69%
Epoch [2/10], Training Loss: 0.766, Validation Accuracy: 61.84%
Epoch [3/10], Training Loss: 0.714, Validation Accuracy: 62.00%
Epoch [4/10], Training Loss: 0.681, Validation Accuracy: 62.58%
Epoch [5/10], Training Loss: 0.637, Validation Accuracy: 61.48%
Epoch [6/10], Training Loss: 0.607, Validation Accuracy: 62.14%
Epoch [7/10], Training Loss: 0.578, Validation Accuracy: 62.17%
Epoch [8/10], Training Loss: 0.555, Validation Accuracy: 62.04%
Epoch [9/10], Training Loss: 0.549, Validation Accuracy: 61.94%
Epoch [10/10], Training Loss: 0.519, Validation Accuracy: 61.31%
Epoch [1/10], Training Loss: 0.824, Validation Accuracy: 61.41%
Epoch [2/10], Training Loss: 0.720, Validation Accuracy: 62.61%
Epoch [3/10], Training Loss: 0.661, Validation Accuracy: 62.41%
Epoch [4/10], Training Loss: 0.622, Validation Accuracy: 62.09%
Epoch [5/10], Training Loss: 0.594, Validation Accuracy: 62.43%
Epoch [6/10], Training Loss: 0.566, Validation Accuracy: 62.50%
Epoch [7/10], Training Loss: 0.539, Validation Accuracy: 62.33%
Epoch [8/10], Training Loss: 0.515, Validation Accuracy: 62.54%
Epoch [9/10], Training Loss: 0.498, Validation Accuracy: 62.14%
Epoch [10/10], Training Loss: 0.474, Validation Accuracy: 62.02%
Epoch [1/10], Training Loss: 0.813, Validation Accuracy: 62.09%
Epoch [2/10], Training Loss: 0.704, Validation Accuracy: 62.24%
Epoch [3/10], Training Loss: 0.650, Validation Accuracy: 61.97%
Epoch [4/10], Training Loss: 0.615, Validation Accuracy: 61.10%
Epoch [5/10], Training Loss: 0.582, Validation Accuracy: 61.58%
Epoch [6/10], Training Loss: 0.544, Validation Accuracy: 62.18%
Epoch [7/10], Training Loss: 0.527, Validation Accuracy: 61.91%
Epoch [8/10], Training Loss: 0.495, Validation Accuracy: 62.12%
Epoch [9/10], Training Loss: 0.479, Validation Accuracy: 62.02%
Epoch [10/10], Training Loss: 0.456, Validation Accuracy: 61.96%
Epoch [1/10], Training Loss: 0.833, Validation Accuracy: 60.07%
Epoch [2/10], Training Loss: 0.709, Validation Accuracy: 62.37%
Epoch [3/10], Training Loss: 0.641, Validation Accuracy: 62.13%
Epoch [4/10], Training Loss: 0.593, Validation Accuracy: 62.20%
Epoch [5/10], Training Loss: 0.564, Validation Accuracy: 62.38%
Epoch [6/10], Training Loss: 0.542, Validation Accuracy: 62.29%
Epoch [7/10], Training Loss: 0.509, Validation Accuracy: 62.03%
Epoch [8/10], Training Loss: 0.480, Validation Accuracy: 62.12%
Epoch [9/10], Training Loss: 0.463, Validation Accuracy: 61.53%
Epoch [10/10], Training Loss: 0.442, Validation Accuracy: 61.38%
Epoch [1/10], Training Loss: 0.836, Validation Accuracy: 61.01%
Epoch [2/10], Training Loss: 0.711, Validation Accuracy: 61.97%
Epoch [3/10], Training Loss: 0.638, Validation Accuracy: 61.92%
Epoch [4/10], Training Loss: 0.592, Validation Accuracy: 61.95%
Epoch [5/10], Training Loss: 0.567, Validation Accuracy: 61.84%
Epoch [6/10], Training Loss: 0.525, Validation Accuracy: 61.86%
Epoch [7/10], Training Loss: 0.495, Validation Accuracy: 62.05%
Epoch [8/10], Training Loss: 0.483, Validation Accuracy: 61.44%
Epoch [9/10], Training Loss: 0.458, Validation Accuracy: 61.32%
Epoch [10/10], Training Loss: 0.433, Validation Accuracy: 61.93%
Epoch [1/10], Training Loss: 0.831, Validation Accuracy: 61.10%
Epoch [2/10], Training Loss: 0.693, Validation Accuracy: 61.29%
Epoch [3/10], Training Loss: 0.638, Validation Accuracy: 61.46%
Epoch [4/10], Training Loss: 0.594, Validation Accuracy: 62.03%
Epoch [5/10], Training Loss: 0.555, Validation Accuracy: 61.52%
Epoch [6/10], Training Loss: 0.523, Validation Accuracy: 61.88%
Epoch [7/10], Training Loss: 0.494, Validation Accuracy: 61.80%
Epoch [8/10], Training Loss: 0.472, Validation Accuracy: 60.82%
Epoch [9/10], Training Loss: 0.462, Validation Accuracy: 61.37%
Epoch [10/10], Training Loss: 0.432, Validation Accuracy: 60.73%
Epoch [1/10], Training Loss: 0.810, Validation Accuracy: 61.07%
Epoch [2/10], Training Loss: 0.676, Validation Accuracy: 61.70%
Epoch [3/10], Training Loss: 0.604, Validation Accuracy: 61.66%
Epoch [4/10], Training Loss: 0.551, Validation Accuracy: 62.01%
Epoch [5/10], Training Loss: 0.510, Validation Accuracy: 62.14%
Epoch [6/10], Training Loss: 0.484, Validation Accuracy: 62.45%
Epoch [7/10], Training Loss: 0.456, Validation Accuracy: 62.15%
Epoch [8/10], Training Loss: 0.428, Validation Accuracy: 62.43%
Epoch [9/10], Training Loss: 0.404, Validation Accuracy: 61.46%
Epoch [10/10], Training Loss: 0.385, Validation Accuracy: 62.23%
"""

# Regular expression to find validation accuracies
accuracies = re.findall(r'Validation Accuracy: (\d+\.\d+)%', log)

# Convert accuracies from string to float
accuracies = [float(acc) for acc in accuracies]

# Print accuracies
print("Accuracies:", accuracies)

# Print size of the array
print("Size of array:", len(accuracies))


Accuracies: [10.63, 10.65, 12.53, 15.01, 15.84, 16.61, 17.22, 17.73, 18.03, 18.24, 18.29, 18.64, 19.14, 20.29, 22.04, 23.48, 24.06, 24.25, 25.54, 25.73, 26.48, 27.31, 27.95, 27.97, 28.7, 29.93, 30.11, 30.36, 31.1, 31.93, 32.49, 33.43, 33.96, 34.24, 35.3, 36.12, 35.8, 36.71, 37.0, 37.5, 37.77, 38.55, 38.23, 39.57, 39.43, 40.0, 40.53, 40.64, 41.21, 41.65, 41.89, 41.41, 42.37, 42.68, 42.57, 43.18, 43.22, 43.7, 44.55, 44.1, 44.37, 45.17, 44.57, 45.02, 44.36, 44.68, 45.39, 45.88, 44.94, 46.33, 46.75, 46.79, 47.1, 46.53, 46.49, 46.99, 47.39, 46.19, 47.79, 47.39, 47.94, 47.87, 48.2, 48.09, 48.6, 48.58, 48.44, 49.38, 49.27, 48.94, 49.25, 50.23, 49.71, 50.06, 50.22, 50.55, 50.25, 50.75, 50.64, 51.1, 50.93, 49.48, 51.56, 51.36, 51.61, 51.57, 51.84, 50.76, 51.94, 52.48, 51.3, 51.24, 52.84, 52.46, 52.85, 52.53, 52.29, 52.66, 52.52, 52.7, 52.43, 53.74, 53.61, 53.54, 52.69, 53.28, 52.36, 51.92, 53.71, 54.17, 52.43, 52.7, 54.28, 54.36, 54.34, 54.22, 54.97, 54.17, 54.32, 55.15, 54.46, 55.0, 54.94, 55.

In [14]:
!git clone https://github.com/Rahad31/Kl-FedDis-Research-.git


Cloning into 'Kl-FedDis-Research-'...


In [15]:
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/Rahad31/Kl-FedDis-Research-.git
!git push -u origin main

Reinitialized existing Git repository in /content/Kl-FedDis-Research-/.git/
fatal: pathspec 'README.md' did not match any files
On branch main

Initial commit

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Kl-FedDis-Research-/

nothing added to commit but untracked files present (use "git add" to track)
error: remote origin already exists.
error: src refspec main does not match any
error: failed to push some refs to 'https://github.com/Rahad31/Kl-FedDis-Research-.git'
